In [1]:
!pip install python-constraint

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24084 sha256=6c0e7748fc89f2463fb09317ca5bab66deccb6545e32848dbb3cc2bda536653f
  Stored in directory: c:\users\omar amir\appdata\local\pip\cache\wheels\c1\d2\3d\082849b61a9c6de02d4a7c8a402c224640f08d8a971307b92b
Successfully built python-constraint


In [8]:
# Example data
subjects = [
    {
        'subject_id': 'CS101',
        'subject_name': 'Introduction to Programming',
        'students': 45,
        'professor': 'Smith',
        'required_room_type': 'lab'
    },
    {
        'subject_id': 'CS102', 
        'subject_name': 'Data Structures',
        'students': 40,
        'professor': 'Smith',  # Same professor - will trigger constraint
        'required_room_type': 'lecture_hall'
    },
    {
        'subject_id': 'MATH101',
        'subject_name': 'Calculus',
        'students': 60,
        'professor': 'Johnson',
        'preferred_times': ['Mon_9-11', 'Wed_9-11']  # Specific time constraint
    }
]

rooms = [
    {'room_id': 'A101', 'capacity': 50, 'room_type': 'lab'},
    {'room_id': 'A102', 'capacity': 50, 'room_type': 'lab'},
    {'room_id': 'B201', 'capacity': 70, 'room_type': 'lecture_hall'},
    {'room_id': 'B202', 'capacity': 80, 'room_type': 'lecture_hall'}
]

time_slots = ['Mon_9-11', 'Mon_11-1', 'Mon_2-4', 'Tue_9-11', 'Tue_11-1', 'Tue_2-4']

In [9]:
from constraint import Problem

def create_basic_scheduler(subjects, rooms, time_slots):
    problem = Problem()
    
    # Add variables (each subject needs a time slot and room)
    for subject in subjects:
        # Filter rooms by capacity for this subject
        suitable_rooms = [room['room_id'] for room in rooms 
                         if room['capacity'] >= subject['students']]
        
        problem.addVariable(
            f"{subject['subject_id']}_time", 
            time_slots
        )
        problem.addVariable(
            f"{subject['subject_id']}_room", 
            suitable_rooms
        )
    
    # CONSTRAINT 1: No professor double-booking
    professor_subjects = {}
    for subject in subjects:
        prof = subject['professor']
        if prof not in professor_subjects:
            professor_subjects[prof] = []
        professor_subjects[prof].append(subject['subject_id'])
    
    for prof, subject_ids in professor_subjects.items():
        if len(subject_ids) > 1:  # Only need constraint if professor teaches multiple subjects
            for i in range(len(subject_ids)):
                for j in range(i + 1, len(subject_ids)):
                    subj1 = subject_ids[i]
                    subj2 = subject_ids[j]
                    problem.addConstraint(
                        lambda time1, time2: time1 != time2,
                        [f"{subj1}_time", f"{subj2}_time"]
                    )
    
    # CONSTRAINT 2: No room double-booking
    # For each room, ensure no two subjects are scheduled at same time
    room_subjects = {}
    for subject in subjects:
        for room in [r for r in rooms if r['capacity'] >= subject['students']]:
            room_id = room['room_id']
            if room_id not in room_subjects:
                room_subjects[room_id] = []
            room_subjects[room_id].append(subject['subject_id'])
    
    for room_id, subject_ids in room_subjects.items():
        if len(subject_ids) > 1:
            for i in range(len(subject_ids)):
                for j in range(i + 1, len(subject_ids)):
                    subj1 = subject_ids[i]
                    subj2 = subject_ids[j]
                    problem.addConstraint(
                        lambda time1, time2, room1, room2: 
                            not (room1 == room_id and room2 == room_id and time1 == time2),
                        [f"{subj1}_time", f"{subj2}_time", f"{subj1}_room", f"{subj2}_room"]
                    )
    
    # CONSTRAINT 3: Room type constraints (if you have specific room requirements)
    for subject in subjects:
        if 'required_room_type' in subject:
            problem.addConstraint(
                lambda room_id, subj_id=subject['subject_id']: 
                    any(room['room_type'] == subject['required_room_type'] 
                        for room in rooms if room['room_id'] == room_id),
                [f"{subject['subject_id']}_room"]
            )
    
    # CONSTRAINT 4: Specific time constraints (e.g., some subjects only in morning)
    for subject in subjects:
        if 'preferred_times' in subject:
            problem.addConstraint(
                lambda time: time in subject['preferred_times'],
                [f"{subject['subject_id']}_time"]
            )
    
    return problem

In [10]:
def solve_with_fallback():
    problem = create_basic_scheduler(subjects, rooms, time_slots)
    
    solution = problem.getSolution()
    if solution:
        return solution
    
    # If no solution, try relaxing constraints
    print("No solution found. Trying to relax constraints...")
    
    # Remove room type constraints
    for subject in subjects:
        problem.removeConstraint(f"{subject['subject_id']}_room")
    
    solution = problem.getSolution()
    return solution

In [11]:
# CONSTRAINT 5: Minimum gap between classes for same professor
def add_time_gap_constraint(problem, subject1_id, subject2_id, min_gap_hours):
    problem.addConstraint(
        lambda time1, time2: abs(time_slots.index(time1) - time_slots.index(time2)) >= min_gap_hours,
        [f"{subject1_id}_time", f"{subject2_id}_time"]
    )

# CONSTRAINT 6: Subjects that must be on same day
def add_same_day_constraint(problem, subject1_id, subject2_id):
    problem.addConstraint(
        lambda time1, time2: time1.split('_')[0] == time2.split('_')[0],
        [f"{subject1_id}_time", f"{subject2_id}_time"]
    )

# CONSTRAINT 7: Limit classes per day for a professor
def add_max_classes_per_day(problem, professor, max_classes):
    prof_subjects = [s for s in subjects if s['professor'] == professor]
    subject_ids = [s['subject_id'] for s in prof_subjects]
    
    # This is more complex - would need a custom constraint function
    def max_classes_constraint(*times):
        days = [t.split('_')[0] for t in times]
        from collections import Counter
        day_counts = Counter(days)
        return all(count <= max_classes for count in day_counts.values())
    
    problem.addConstraint(
        max_classes_constraint,
        [f"{subj_id}_time" for subj_id in subject_ids]
    )

In [20]:
from constraint import Problem

def create_basic_scheduler(subjects, rooms, time_slots):
    problem = Problem()
    
    # Add variables (each subject needs a time slot and room)
    for subject in subjects:
        # Filter rooms by capacity for this subject
        suitable_rooms = [room['room_id'] for room in rooms 
                         if room['capacity'] >= subject['students']]
        
        problem.addVariable(
            f"{subject['subject_id']}_time", 
            time_slots
        )
        problem.addVariable(
            f"{subject['subject_id']}_room", 
            suitable_rooms
        )
    
    # CONSTRAINT 1: No professor double-booking
    professor_subjects = {}
    for subject in subjects:
        prof = subject['professor']
        if prof not in professor_subjects:
            professor_subjects[prof] = []
        professor_subjects[prof].append(subject['subject_id'])
    
    for prof, subject_ids in professor_subjects.items():
        if len(subject_ids) > 1:
            for i in range(len(subject_ids)):
                for j in range(i + 1, len(subject_ids)):
                    subj1 = subject_ids[i]
                    subj2 = subject_ids[j]
                    problem.addConstraint(
                        lambda time1, time2: time1 != time2,
                        [f"{subj1}_time", f"{subj2}_time"]
                    )
    
    # CONSTRAINT 2: No room double-booking (simplified approach)
    # Get all subject combinations
    subject_ids = [s['subject_id'] for s in subjects]
    for i in range(len(subject_ids)):
        for j in range(i + 1, len(subject_ids)):
            subj1 = subject_ids[i]
            subj2 = subject_ids[j]
            problem.addConstraint(
                lambda time1, time2, room1, room2: 
                    not (time1 == time2 and room1 == room2),
                [f"{subj1}_time", f"{subj2}_time", f"{subj1}_room", f"{subj2}_room"]
            )
    
    # CONSTRAINT 3: Room type constraints (with safe access)
    for subject in subjects:
        if subject.get('required_room_type'):  # Safe way to check
            required_type = subject['required_room_type']
            problem.addConstraint(
                lambda room_id, rt=required_type: 
                    any(room['room_type'] == rt for room in rooms if room['room_id'] == room_id),
                [f"{subject['subject_id']}_room"]
            )
    
    # CONSTRAINT 4: Specific time constraints (with safe access)
    for subject in subjects:
        if subject.get('preferred_times'):
            preferred_times = subject['preferred_times']
            problem.addConstraint(
                lambda time, pt=preferred_times: time in pt,
                [f"{subject['subject_id']}_time"]
            )
    
    return problem
    # CONSTRAINT 5: Minimum gap between classes for same professor
    def add_time_gap_constraint(problem, subject1_id, subject2_id, min_gap_hours):
        problem.addConstraint(
            lambda time1, time2: abs(time_slots.index(time1) - time_slots.index(time2)) >= min_gap_hours,
            [f"{subject1_id}_time", f"{subject2_id}_time"]
        )
    
    # CONSTRAINT 6: Subjects that must be on same day
    def add_same_day_constraint(problem, subject1_id, subject2_id):
        problem.addConstraint(
            lambda time1, time2: time1.split('_')[0] == time2.split('_')[0],
            [f"{subject1_id}_time", f"{subject2_id}_time"]
        )
    
    # CONSTRAINT 7: Limit classes per day for a professor
    def add_max_classes_per_day(problem, professor, max_classes):
        prof_subjects = [s for s in subjects if s['professor'] == professor]
        subject_ids = [s['subject_id'] for s in prof_subjects]
        
        # This is more complex - would need a custom constraint function
        def max_classes_constraint(*times):
            days = [t.split('_')[0] for t in times]
            from collections import Counter
            day_counts = Counter(days)
            return all(count <= max_classes for count in day_counts.values())
        
        problem.addConstraint(
            max_classes_constraint,
            [f"{subj_id}_time" for subj_id in subject_ids]
        )
# Updated sample data with safe structure
subjects = [
    {
        'subject_id': 'CS101',
        'subject_name': 'Introduction to Programming',
        'students': 45,
        'professor': 'Dr. Smith',
        'required_room_type': 'lab'  # Now all subjects have this key
    },
    {
        'subject_id': 'CS102', 
        'subject_name': 'Data Structures',
        'students': 120,
        'professor': 'Dr. Smith',
        'required_room_type': 'lecture_hall'
    },
    {
        'subject_id': 'MATH101',
        'subject_name': 'Calculus',
        'students': 120,
        'professor': 'Dr. Johnson',
        'required_room_type': 'lecture_hall',  # Added this
        'preferred_times': ['Mon_9-11', 'Wed_9-11']
    }
]

rooms = [
    {'room_id': '2205', 'capacity': 50, 'room_type': 'lab'},
    {'room_id': '2103', 'capacity': 50, 'room_type': 'lab'},
    {'room_id': '1220', 'capacity': 200, 'room_type': 'lecture_hall'},
    {'room_id': '3214', 'capacity': 200, 'room_type': 'lecture_hall'}
]

time_slots = [
    'Sun_9:00-10:30', 'Sun_10:30-12:00', 'Sun_12:00-13:30', 'Sun_13:30-15:00',
    'Mon_9:00-10:30', 'Mon_10:30-12:00', 'Mon_12:00-13:30', 'Mon_13:30-15:00',
    'Tue_9:00-10:30', 'Tue_10:30-12:00', 'Tue_12:00-13:30', 'Tue_13:30-15:00',
    'Wed_9:00-10:30', 'Wed_10:30-12:00', 'Wed_12:00-13:30', 'Wed_13:30-15:00',
    'Thu_9:00-10:30', 'Thu_10:30-12:00', 'Thu_12:00-13:30', 'Thu_13:30-15:00'
]

def solve_schedule():
    problem = create_basic_scheduler(subjects, rooms, time_slots)
    
    # Try to get one solution first
    solution = problem.getSolution()
    
    return solution

def print_schedule(solution):
    if not solution:
        print("No solution found!")
        return
    
    print("Generated Schedule:")
    print("-" * 50)
    
    # Group by time slot for better display
    schedule_by_time = {}
    for key, value in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            room_key = f"{subject_id}_room"
            room = solution[room_key]
            
            if value not in schedule_by_time:
                schedule_by_time[value] = []
            schedule_by_time[value].append((subject_id, room))
    
    # Print schedule
    for time_slot in sorted(schedule_by_time.keys()):
        print(f"\n{time_slot}:")
        for subject_id, room in schedule_by_time[time_slot]:
            subject = next(s for s in subjects if s['subject_id'] == subject_id)
            print(f"  {subject['subject_name']} - Room {room} - {subject['professor']}")

# Run it with error handling
try:
    solution = solve_schedule()
    if solution:
        print_schedule(solution)
    else:
        print("No feasible schedule found with given constraints!")
        
        # Debug: Check if rooms are suitable
        print("\nDebug Info:")
        for subject in subjects:
            suitable_rooms = [room['room_id'] for room in rooms 
                           if room['capacity'] >= subject['students']]
            print(f"{subject['subject_id']} needs {subject['students']} seats. Suitable rooms: {suitable_rooms}")
            
except Exception as e:
    print(f"Error: {e}")
    print("\nTroubleshooting tips:")
    print("1. Check that all subjects have 'required_room_type' key")
    print("2. Check that room capacities are sufficient")
    print("3. Check that time_slots are properly formatted")

No feasible schedule found with given constraints!

Debug Info:
CS101 needs 45 seats. Suitable rooms: ['2205', '2103', '1220', '3214']
CS102 needs 120 seats. Suitable rooms: ['1220', '3214']
MATH101 needs 120 seats. Suitable rooms: ['1220', '3214']


In [23]:
from constraint import Problem

# Your utility functions
def get_slot_duration(slot):
    """Calculate duration of a time slot in hours"""
    day, time_range = slot.split('_')
    start, end = time_range.split('-')
    
    # Parse times
    start_h, start_m = map(int, start.split(':')) if ':' in start else (int(start), 0)
    end_h, end_m = map(int, end.split(':')) if ':' in end else (int(end), 0)
    
    duration = (end_h + end_m/60) - (start_h + start_m/60)
    return duration

def get_available_days(time_slots):
    """Get unique days from time slots"""
    return list(set(slot.split('_')[0] for slot in time_slots))

def generate_time_slots(days=['Sun', 'Mon', 'Tue', 'Wed', 'Thu']):
    """Generate time slots for your college schedule"""
    time_slots = []
    start_hour = 9.0  # 9:00 AM
    end_hour = 15.0   # 3:00 PM
    slot_duration = 1.5  # 1.5 hours
    
    num_slots = int((end_hour - start_hour) / slot_duration)
    
    for day in days:
        current_time = start_hour
        for i in range(num_slots):
            start_time = current_time
            end_time = current_time + slot_duration
            
            # Format times
            start_str = format_time(start_time)
            end_str = format_time(end_time)
            
            time_slot = f"{day}_{start_str}-{end_str}"
            time_slots.append(time_slot)
            
            current_time = end_time
    
    return time_slots

def format_time(hour_float):
    """Convert 9.5 to '9:30' format"""
    hours = int(hour_float)
    minutes = int((hour_float - hours) * 60)
    return f"{hours}:{minutes:02d}"

# Main scheduler function with integrated utilities
def create_basic_scheduler(subjects, rooms):
    """Create scheduler with dynamic time slot generation"""
    # Generate time slots dynamically
    time_slots = generate_time_slots()
    
    print(f"Generated {len(time_slots)} time slots across {len(get_available_days(time_slots))} days")
    print(f"Each slot duration: {get_slot_duration(time_slots[0])} hours")
    
    problem = Problem()
    
    # Add variables (each subject needs a time slot and room)
    for subject in subjects:
        # Filter rooms by capacity for this subject
        suitable_rooms = [room['room_id'] for room in rooms 
                         if room['capacity'] >= subject['students']]
        
        problem.addVariable(
            f"{subject['subject_id']}_time", 
            time_slots
        )
        problem.addVariable(
            f"{subject['subject_id']}_room", 
            suitable_rooms
        )
    
    # CONSTRAINT 1: No professor double-booking
    professor_subjects = {}
    for subject in subjects:
        prof = subject['professor']
        if prof not in professor_subjects:
            professor_subjects[prof] = []
        professor_subjects[prof].append(subject['subject_id'])
    
    for prof, subject_ids in professor_subjects.items():
        if len(subject_ids) > 1:
            for i in range(len(subject_ids)):
                for j in range(i + 1, len(subject_ids)):
                    subj1 = subject_ids[i]
                    subj2 = subject_ids[j]
                    problem.addConstraint(
                        lambda time1, time2: time1 != time2,
                        [f"{subj1}_time", f"{subj2}_time"]
                    )
    
    # CONSTRAINT 2: No room double-booking
    subject_ids = [s['subject_id'] for s in subjects]
    for i in range(len(subject_ids)):
        for j in range(i + 1, len(subject_ids)):
            subj1 = subject_ids[i]
            subj2 = subject_ids[j]
            problem.addConstraint(
                lambda time1, time2, room1, room2: 
                    not (time1 == time2 and room1 == room2),
                [f"{subj1}_time", f"{subj2}_time", f"{subj1}_room", f"{subj2}_room"]
            )
    
    # CONSTRAINT 3: Room type constraints
    for subject in subjects:
        if subject.get('required_room_type'):
            required_type = subject['required_room_type']
            problem.addConstraint(
                lambda room_id, rt=required_type: 
                    any(room['room_type'] == rt for room in rooms if room['room_id'] == room_id),
                [f"{subject['subject_id']}_room"]
            )
    
    return problem, time_slots  # Return time_slots for use elsewhere

# Enhanced schedule printing with duration info
def print_schedule(solution, subjects, time_slots):
    if not solution:
        print("No solution found!")
        return
    
    print("\n" + "="*60)
    print("GENERATED SCHEDULE")
    print("="*60)
    
    # Group by day for better organization
    schedule_by_day = {}
    for key, value in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            room_key = f"{subject_id}_room"
            room = solution[room_key]
            day = value.split('_')[0]
            
            if day not in schedule_by_day:
                schedule_by_day[day] = []
            
            subject_info = next(s for s in subjects if s['subject_id'] == subject_id)
            schedule_by_day[day].append({
                'time_slot': value,
                'subject': subject_info,
                'room': room,
                'duration': get_slot_duration(value)
            })
    
    # Print schedule organized by day
    for day in sorted(schedule_by_day.keys()):
        print(f"\n{day.upper()}:")
        print("-" * 40)
        
        # Sort by time
        day_schedule = sorted(schedule_by_day[day], 
                             key=lambda x: x['time_slot'])
        
        for entry in day_schedule:
            subject = entry['subject']
            print(f"  {entry['time_slot'].split('_')[1]:<11} | "
                  f"Room: {entry['room']:<6} | "
                  f"{subject['professor']:<12} | "
                  f"{subject['subject_name']}")

# Sample data
subjects = [
    {
        'subject_id': 'CS101',
        'subject_name': 'Introduction to Programming',
        'students': 45,
        'professor': 'Dr. Smith',
        'required_room_type': 'lab'
    },
    {
        'subject_id': 'CS102', 
        'subject_name': 'Data Structures',
        'students': 40,
        'professor': 'Dr. Smith',
        'required_room_type': 'lecture_hall'
    },
    {
        'subject_id': 'MATH101',
        'subject_name': 'Calculus',
        'students': 100,
        'professor': 'Dr. Johnson',
        'required_room_type': 'lecture_hall'
    }
]

rooms = [
    {'room_id': 'A101', 'capacity': 50, 'room_type': 'lab'},
    {'room_id': 'A102', 'capacity': 50, 'room_type': 'lab'},
    {'room_id': 'B201', 'capacity': 200, 'room_type': 'lecture_hall'},
    {'room_id': 'B202', 'capacity': 200, 'room_type': 'lecture_hall'}
]

# Main execution
def main():
    print("University Schedule Generator")
    print("Generating time slots...")
    
    # Create scheduler with integrated time slots
    problem, time_slots = create_basic_scheduler(subjects, rooms)
    
    # Display generated time slots
    print(f"\nAvailable Time Slots ({len(time_slots)} total):")
    days = get_available_days(time_slots)
    for day in sorted(days):
        day_slots = [slot for slot in time_slots if slot.startswith(day)]
        print(f"  {day}: {len(day_slots)} slots")
    
    # Solve the schedule
    print("\nFinding optimal schedule...")
    solution = problem.getSolution()
    
    if solution:
        print_schedule(solution, subjects, time_slots)
        
        # Additional statistics
        total_hours = sum(get_slot_duration(slot) for slot in solution.values() 
                         if isinstance(slot, str) and '_' in slot)
        print(f"\nSchedule Statistics:")
        print(f"Total instructional hours: {total_hours}")
        print(f"Subjects scheduled: {len(subjects)}")
        print(f"Time slots utilized: {len(set(solution.values()))}")
        
    else:
        print("No feasible schedule found!")
        # Debug information
        print("\nDebug Info:")
        for subject in subjects:
            suitable_rooms = [room['room_id'] for room in rooms 
                           if room['capacity'] >= subject['students']]
            print(f"{subject['subject_id']}: needs {subject['students']} seats, "
                  f"suitable rooms: {suitable_rooms}")

# Run the program
if __name__ == "__main__":
    main()

University Schedule Generator
Generating time slots...
Generated 20 time slots across 5 days
Each slot duration: 1.5 hours

Available Time Slots (20 total):
  Mon: 4 slots
  Sun: 4 slots
  Thu: 4 slots
  Tue: 4 slots
  Wed: 4 slots

Finding optimal schedule...

GENERATED SCHEDULE

THU:
----------------------------------------
  12:00-13:30 | Room: B202   | Dr. Smith    | Data Structures
  13:30-15:00 | Room: A102   | Dr. Smith    | Introduction to Programming
  13:30-15:00 | Room: B201   | Dr. Johnson  | Calculus

Schedule Statistics:
Total instructional hours: 4.5
Subjects scheduled: 3
Time slots utilized: 5


In [24]:
from constraint import Problem

def create_basic_scheduler(subjects, rooms, professor_day_off):
    """Create scheduler with day off constraints"""
    time_slots = generate_time_slots()
    
    problem = Problem()
    
    # Add variables (each subject needs a time slot and room)
    for subject in subjects:
        suitable_rooms = [room['room_id'] for room in rooms 
                         if room['capacity'] >= subject['students']]
        
        problem.addVariable(f"{subject['subject_id']}_time", time_slots)
        problem.addVariable(f"{subject['subject_id']}_room", suitable_rooms)
    
    # CONSTRAINT 1: No professor double-booking
    professor_subjects = {}
    for subject in subjects:
        prof = subject['professor']
        if prof not in professor_subjects:
            professor_subjects[prof] = []
        professor_subjects[prof].append(subject['subject_id'])
    
    for prof, subject_ids in professor_subjects.items():
        if len(subject_ids) > 1:
            for i in range(len(subject_ids)):
                for j in range(i + 1, len(subject_ids)):
                    subj1 = subject_ids[i]
                    subj2 = subject_ids[j]
                    problem.addConstraint(
                        lambda time1, time2: time1 != time2,
                        [f"{subj1}_time", f"{subj2}_time"]
                    )
    
    # CONSTRAINT 2: No room double-booking
    subject_ids = [s['subject_id'] for s in subjects]
    for i in range(len(subject_ids)):
        for j in range(i + 1, len(subject_ids)):
            subj1 = subject_ids[i]
            subj2 = subject_ids[j]
            problem.addConstraint(
                lambda time1, time2, room1, room2: 
                    not (time1 == time2 and room1 == room2),
                [f"{subj1}_time", f"{subj2}_time", f"{subj1}_room", f"{subj2}_room"]
            )
    
    # CONSTRAINT 3: Room type constraints
    for subject in subjects:
        if subject.get('required_room_type'):
            required_type = subject['required_room_type']
            problem.addConstraint(
                lambda room_id, rt=required_type: 
                    any(room['room_type'] == rt for room in rooms if room['room_id'] == room_id),
                [f"{subject['subject_id']}_room"]
            )
    
    # NEW CONSTRAINT 4: Professor day off constraints
    for subject in subjects:
        prof = subject['professor']
        if prof in professor_day_off:
            day_off = professor_day_off[prof]
            problem.addConstraint(
                lambda time_slot, off_day=day_off: not time_slot.startswith(off_day),
                [f"{subject['subject_id']}_time"]
            )
    
    return problem, time_slots

# Updated sample data with day off information
subjects = [
    {
        'subject_id': 'CS101',
        'subject_name': 'Introduction to Programming',
        'students': 45,
        'professor': 'Dr. Smith',
        'required_room_type': 'lab'
    },
    {
        'subject_id': 'CS102', 
        'subject_name': 'Data Structures',
        'students': 40,
        'professor': 'Dr. Smith',  # Same professor - has day off on Sunday
        'required_room_type': 'lecture_hall'
    },
    {
        'subject_id': 'MATH101',
        'subject_name': 'Calculus',
        'students': 60,
        'professor': 'Dr. Johnson',  # Day off on Monday
        'required_room_type': 'lecture_hall'
    },
    {
        'subject_id': 'PHY101',
        'subject_name': 'Physics',
        'students': 50,
        'professor': 'Dr. Brown',  # No day off specified
        'required_room_type': 'lab'
    }
]

# Professor day off information
professor_day_off = {
    'Dr. Smith': 'Sun',    # Dr. Smith doesn't work on Sundays
    'Dr. Johnson': 'Mon',  # Dr. Johnson doesn't work on Mondays
    # Dr. Brown has no day off specified (works all days)
}

rooms = [
    {'room_id': 'A101', 'capacity': 50, 'room_type': 'lab'},
    {'room_id': 'A102', 'capacity': 50, 'room_type': 'lab'},
    {'room_id': 'B201', 'capacity': 70, 'room_type': 'lecture_hall'},
    {'room_id': 'B202', 'capacity': 80, 'room_type': 'lecture_hall'}
]

# Enhanced printing to show day off information
def print_schedule_with_day_off(solution, subjects, time_slots, professor_day_off):
    if not solution:
        print("No solution found!")
        return
    
    print("\n" + "="*70)
    print("GENERATED SCHEDULE (with Professor Day Off Constraints)")
    print("="*70)
    
    # Display professor day off information
    print("\nProfessor Day Off Information:")
    for prof, day_off in professor_day_off.items():
        print(f"  {prof}: Day off on {day_off}")
    if any(prof not in professor_day_off for prof in set(s['professor'] for s in subjects)):
        print("  Note: Some professors work all days")
    
    # Group by day for better organization
    schedule_by_day = {}
    for key, value in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            room_key = f"{subject_id}_room"
            room = solution[room_key]
            day = value.split('_')[0]
            
            if day not in schedule_by_day:
                schedule_by_day[day] = []
            
            subject_info = next(s for s in subjects if s['subject_id'] == subject_id)
            schedule_by_day[day].append({
                'time_slot': value,
                'subject': subject_info,
                'room': room,
                'duration': get_slot_duration(value)
            })
    
    # Print schedule organized by day
    for day in sorted(schedule_by_day.keys()):
        print(f"\n{day.upper()}:")
        print("-" * 50)
        
        # Sort by time
        day_schedule = sorted(schedule_by_day[day], key=lambda x: x['time_slot'])
        
        for entry in day_schedule:
            subject = entry['subject']
            prof = subject['professor']
            
            # Check if this professor should be working on this day
            day_off_status = ""
            if prof in professor_day_off and professor_day_off[prof] == day:
                day_off_status = " ⚠️  CONFLICT - Professor Day Off!"
            elif prof in professor_day_off:
                day_off_status = " ✅ Professor Available"
            else:
                day_off_status = " ✅ No Day Off Specified"
            
            print(f"  {entry['time_slot'].split('_')[1]:<11} | "
                  f"Room: {entry['room']:<6} | "
                  f"Dr. {prof:<12} | "
                  f"{subject['subject_name']:<30} {day_off_status}")

# Utility functions (same as before)
def get_slot_duration(slot):
    day, time_range = slot.split('_')
    start, end = time_range.split('-')
    start_h, start_m = map(int, start.split(':')) if ':' in start else (int(start), 0)
    end_h, end_m = map(int, end.split(':')) if ':' in end else (int(end), 0)
    duration = (end_h + end_m/60) - (start_h + start_m/60)
    return duration

def get_available_days(time_slots):
    return list(set(slot.split('_')[0] for slot in time_slots))

def generate_time_slots(days=['Sun', 'Mon', 'Tue', 'Wed', 'Thu']):
    time_slots = []
    start_hour = 9.0
    end_hour = 15.0
    slot_duration = 1.5
    num_slots = int((end_hour - start_hour) / slot_duration)
    
    for day in days:
        current_time = start_hour
        for i in range(num_slots):
            start_time = current_time
            end_time = current_time + slot_duration
            start_str = format_time(start_time)
            end_str = format_time(end_time)
            time_slot = f"{day}_{start_str}-{end_str}"
            time_slots.append(time_slot)
            current_time = end_time
    
    return time_slots

def format_time(hour_float):
    hours = int(hour_float)
    minutes = int((hour_float - hours) * 60)
    return f"{hours}:{minutes:02d}"

# Main execution with day off constraints
def main():
    print("University Schedule Generator with Professor Day Off Constraints")
    print("="*60)
    
    # Create scheduler with day off constraints
    problem, time_slots = create_basic_scheduler(subjects, rooms, professor_day_off)
    
    # Display information
    print(f"Generated {len(time_slots)} time slots")
    print(f"Professors with day off constraints: {len(professor_day_off)}")
    
    # Solve the schedule
    print("\nFinding optimal schedule considering day off constraints...")
    solution = problem.getSolution()
    
    if solution:
        print_schedule_with_day_off(solution, subjects, time_slots, professor_day_off)
        
        # Validate day off constraints
        print("\n" + "="*70)
        print("VALIDATION REPORT:")
        print("="*70)
        validate_day_off_constraints(solution, subjects, professor_day_off)
        
    else:
        print("No feasible schedule found with the given constraints!")
        print("This might be because:")
        print("- Too many professors have overlapping day off restrictions")
        print("- Not enough available time slots")
        print("- Room constraints are too restrictive")

def validate_day_off_constraints(solution, subjects, professor_day_off):
    """Validate that no professor is scheduled on their day off"""
    violations = []
    
    for key, time_slot in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            subject = next(s for s in subjects if s['subject_id'] == subject_id)
            professor = subject['professor']
            day = time_slot.split('_')[0]
            
            if professor in professor_day_off and professor_day_off[professor] == day:
                violations.append(f"❌ {professor} scheduled on {day} (day off) for {subject['subject_name']}")
    
    if violations:
        print("DAY OFF CONSTRAINT VIOLATIONS FOUND:")
        for violation in violations:
            print(f"  {violation}")
    else:
        print("✅ All day off constraints satisfied!")

# Run the program
if __name__ == "__main__":
    main()

University Schedule Generator with Professor Day Off Constraints
Generated 20 time slots
Professors with day off constraints: 2

Finding optimal schedule considering day off constraints...

GENERATED SCHEDULE (with Professor Day Off Constraints)

Professor Day Off Information:
  Dr. Smith: Day off on Sun
  Dr. Johnson: Day off on Mon
  Note: Some professors work all days

THU:
--------------------------------------------------
  12:00-13:30 | Room: B202   | Dr. Dr. Smith    | Data Structures                 ✅ Professor Available
  12:00-13:30 | Room: A102   | Dr. Dr. Brown    | Physics                         ✅ No Day Off Specified
  13:30-15:00 | Room: A102   | Dr. Dr. Smith    | Introduction to Programming     ✅ Professor Available
  13:30-15:00 | Room: B202   | Dr. Dr. Johnson  | Calculus                        ✅ Professor Available

VALIDATION REPORT:
✅ All day off constraints satisfied!


=====================================================================================================

In [1]:
from constraint import Problem

def get_user_input():
    """Dynamically get all input data from the user"""
    subjects = []
    rooms = []
    
    print("=== University Schedule Generator - Data Input ===")
    print("\nStep 1: Enter Subject Information")
    print("-----------------------------------")
    
    while True:
        print(f"\nSubject #{len(subjects) + 1}:")
        subject_id = input("Subject ID (e.g., CS101): ").strip()
        subject_name = input("Subject Name: ").strip()
        students = int(input("Number of students: "))
        professor = input("Professor name: ").strip()
        required_room_type = input("Required room type (lab/lecture_hall): ").strip()
        day_off = input("Professor's day off (Sun/Mon/Tue/Wed/Thu or leave blank if none): ").strip()
        
        subject = {
            'subject_id': subject_id,
            'subject_name': subject_name,
            'students': students,
            'professor': professor,
            'required_room_type': required_room_type,
            'day_off': day_off if day_off else None  # Store day off with subject
        }
        subjects.append(subject)
        
        more = input("\nAdd another subject? (y/n): ").strip().lower()
        if more != 'y':
            break
    
    print("\nStep 2: Enter Room Information")
    print("-----------------------------------")
    
    while True:
        print(f"\nRoom #{len(rooms) + 1}:")
        room_id = input("Room ID: ").strip()
        capacity = int(input("Room capacity: "))
        room_type = input("Room type (lab/lecture_hall): ").strip()
        
        room = {
            'room_id': room_id,
            'capacity': capacity,
            'room_type': room_type
        }
        rooms.append(room)
        
        more = input("\nAdd another room? (y/n): ").strip().lower()
        if more != 'y':
            break
    
    return subjects, rooms

def create_basic_scheduler(subjects, rooms):
    """Create scheduler with integrated day off constraints"""
    time_slots = generate_time_slots()
    
    problem = Problem()
    
    # Add variables (each subject needs a time slot and room)
    for subject in subjects:
        suitable_rooms = [room['room_id'] for room in rooms 
                         if room['capacity'] >= subject['students']]
        
        problem.addVariable(f"{subject['subject_id']}_time", time_slots)
        problem.addVariable(f"{subject['subject_id']}_room", suitable_rooms)
    
    # CONSTRAINT 1: No professor double-booking
    professor_subjects = {}
    for subject in subjects:
        prof = subject['professor']
        if prof not in professor_subjects:
            professor_subjects[prof] = []
        professor_subjects[prof].append(subject['subject_id'])
    
    for prof, subject_ids in professor_subjects.items():
        if len(subject_ids) > 1:
            for i in range(len(subject_ids)):
                for j in range(i + 1, len(subject_ids)):
                    subj1 = subject_ids[i]
                    subj2 = subject_ids[j]
                    problem.addConstraint(
                        lambda time1, time2: time1 != time2,
                        [f"{subj1}_time", f"{subj2}_time"]
                    )
    
    # CONSTRAINT 2: No room double-booking
    subject_ids = [s['subject_id'] for s in subjects]
    for i in range(len(subject_ids)):
        for j in range(i + 1, len(subject_ids)):
            subj1 = subject_ids[i]
            subj2 = subject_ids[j]
            problem.addConstraint(
                lambda time1, time2, room1, room2: 
                    not (time1 == time2 and room1 == room2),
                [f"{subj1}_time", f"{subj2}_time", f"{subj1}_room", f"{subj2}_room"]
            )
    
    # CONSTRAINT 3: Room type constraints
    for subject in subjects:
        if subject.get('required_room_type'):
            required_type = subject['required_room_type']
            problem.addConstraint(
                lambda room_id, rt=required_type: 
                    any(room['room_type'] == rt for room in rooms if room['room_id'] == room_id),
                [f"{subject['subject_id']}_room"]
            )
    
    # CONSTRAINT 4: Professor day off constraints (now integrated with subject data)
    for subject in subjects:
        if subject.get('day_off'):  # Only add constraint if day_off is specified
            day_off = subject['day_off']
            problem.addConstraint(
                lambda time_slot, off_day=day_off: not time_slot.startswith(off_day),
                [f"{subject['subject_id']}_time"]
            )
    
    return problem, time_slots

def print_schedule(solution, subjects, time_slots):
    """Print the generated schedule with integrated day off information"""
    if not solution:
        print("No solution found!")
        return
    
    print("\n" + "="*70)
    print("GENERATED SCHEDULE")
    print("="*70)
    
    # Display professor day off information from subject data
    professors_with_day_off = {s['professor']: s['day_off'] for s in subjects if s.get('day_off')}
    if professors_with_day_off:
        print("\nProfessor Day Off Information:")
        for prof, day_off in professors_with_day_off.items():
            print(f"  {prof}: Day off on {day_off}")
    
    # Group by day for better organization
    schedule_by_day = {}
    for key, value in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            room_key = f"{subject_id}_room"
            room = solution[room_key]
            day = value.split('_')[0]
            
            if day not in schedule_by_day:
                schedule_by_day[day] = []
            
            subject_info = next(s for s in subjects if s['subject_id'] == subject_id)
            schedule_by_day[day].append({
                'time_slot': value,
                'subject': subject_info,
                'room': room,
                'duration': get_slot_duration(value)
            })
    
    # Print schedule organized by day
    for day in sorted(schedule_by_day.keys()):
        print(f"\n{day.upper()}:")
        print("-" * 50)
        
        # Sort by time
        day_schedule = sorted(schedule_by_day[day], key=lambda x: x['time_slot'])
        
        for entry in day_schedule:
            subject = entry['subject']
            prof = subject['professor']
            day_off = subject.get('day_off')
            
            # Check day off status
            day_off_status = ""
            if day_off and day_off == day:
                day_off_status = " ⚠️  CONFLICT - Professor Day Off!"
            elif day_off:
                day_off_status = " ✅ Professor Available"
            else:
                day_off_status = " ✅ No Day Off Specified"
            
            print(f"  {entry['time_slot'].split('_')[1]:<11} | "
                  f"Room: {entry['room']:<6} | "
                  f"Dr. {prof:<12} | "
                  f"{subject['subject_name']:<30} {day_off_status}")

# Utility functions (unchanged)
def get_slot_duration(slot):
    day, time_range = slot.split('_')
    start, end = time_range.split('-')
    start_h, start_m = map(int, start.split(':')) if ':' in start else (int(start), 0)
    end_h, end_m = map(int, end.split(':')) if ':' in end else (int(end), 0)
    duration = (end_h + end_m/60) - (start_h + start_m/60)
    return duration

def get_available_days(time_slots):
    return list(set(slot.split('_')[0] for slot in time_slots))

def generate_time_slots(days=['Sun', 'Mon', 'Tue', 'Wed', 'Thu']):
    time_slots = []
    start_hour = 9.0
    end_hour = 15.0
    slot_duration = 1.5
    num_slots = int((end_hour - start_hour) / slot_duration)
    
    for day in days:
        current_time = start_hour
        for i in range(num_slots):
            start_time = current_time
            end_time = current_time + slot_duration
            start_str = format_time(start_time)
            end_str = format_time(end_time)
            time_slot = f"{day}_{start_str}-{end_str}"
            time_slots.append(time_slot)
            current_time = end_time
    
    return time_slots

def format_time(hour_float):
    hours = int(hour_float)
    minutes = int((hour_float - hours) * 60)
    return f"{hours}:{minutes:02d}"

def validate_day_off_constraints(solution, subjects):
    """Validate that no professor is scheduled on their day off"""
    violations = []
    
    for key, time_slot in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            subject = next(s for s in subjects if s['subject_id'] == subject_id)
            professor = subject['professor']
            day_off = subject.get('day_off')
            day = time_slot.split('_')[0]
            
            if day_off and day_off == day:
                violations.append(f"❌ {professor} scheduled on {day} (day off) for {subject['subject_name']}")
    
    if violations:
        print("DAY OFF CONSTRAINT VIOLATIONS FOUND:")
        for violation in violations:
            print(f"  {violation}")
    else:
        print("✅ All day off constraints satisfied!")

# Main execution with dynamic input
def main():
    print("=== University Schedule Generator ===")
    print("Now with Dynamic Data Input!")
    print("="*50)
    
    # Get all data from user
    subjects, rooms = get_user_input()
    
    # Display input summary
    print("\n" + "="*50)
    print("INPUT SUMMARY:")
    print("="*50)
    print(f"Subjects: {len(subjects)}")
    for subject in subjects:
        day_off_info = f" (Day off: {subject['day_off']})" if subject.get('day_off') else ""
        print(f"  - {subject['subject_id']}: {subject['subject_name']} by {subject['professor']}{day_off_info}")
    
    print(f"\nRooms: {len(rooms)}")
    for room in rooms:
        print(f"  - {room['room_id']}: {room['capacity']} seats ({room['room_type']})")
    
    # Create scheduler
    problem, time_slots = create_basic_scheduler(subjects, rooms)
    
    # Display scheduling information
    print(f"\nScheduling Information:")
    print(f"  Time slots generated: {len(time_slots)}")
    print(f"  Available days: {', '.join(sorted(get_available_days(time_slots)))}")
    
    # Solve the schedule
    print("\n" + "="*50)
    print("GENERATING SCHEDULE...")
    print("="*50)
    
    solution = problem.getSolution()
    
    if solution:
        print_schedule(solution, subjects, time_slots)
        
        # Validation
        print("\n" + "="*70)
        print("VALIDATION REPORT:")
        print("="*70)
        validate_day_off_constraints(solution, subjects)
        
        # Statistics
        total_hours = sum(get_slot_duration(slot) for slot in solution.values() 
                         if isinstance(slot, str) and '_' in slot)
        print(f"\nSchedule Statistics:")
        print(f"  Total instructional hours: {total_hours}")
        print(f"  Subjects scheduled: {len(subjects)}")
        print(f"  Time slots utilized: {len(set(solution.values()))}")
        
    else:
        print("No feasible schedule found with the given constraints!")
        print("\nPossible reasons:")
        print("- Too many professors have the same day off restrictions")
        print("- Not enough available time slots")
        print("- Room constraints are too restrictive")
        print("- Insufficient room capacity for the number of students")

# Run the program
if __name__ == "__main__":
    main()

=== University Schedule Generator ===
Now with Dynamic Data Input!
=== University Schedule Generator - Data Input ===

Step 1: Enter Subject Information
-----------------------------------

Subject #1:


Subject ID (e.g., CS101):  CS 102
Subject Name:  Computer Archetecture
Number of students:  80
Professor name:  Ahmed magdy
Required room type (lab/lecture_hall):  lecture_hall
Professor's day off (Sun/Mon/Tue/Wed/Thu or leave blank if none):  Monday

Add another subject? (y/n):  n



Step 2: Enter Room Information
-----------------------------------

Room #1:


Room ID:  1220
Room capacity:  200
Room type (lab/lecture_hall):  lecture_hall

Add another room? (y/n):  y



Room #2:


Room ID:  3214
Room capacity:  120
Room type (lab/lecture_hall):  lecture_hall

Add another room? (y/n):  n



INPUT SUMMARY:
Subjects: 1
  - CS 102: Computer Archetecture by Ahmed magdy (Day off: Monday)

Rooms: 2
  - 1220: 200 seats (lecture_hall)
  - 3214: 120 seats (lecture_hall)

Scheduling Information:
  Time slots generated: 20
  Available days: Mon, Sun, Thu, Tue, Wed

GENERATING SCHEDULE...

GENERATED SCHEDULE

Professor Day Off Information:
  Ahmed magdy: Day off on Monday

THU:
--------------------------------------------------
  13:30-15:00 | Room: 3214   | Dr. Ahmed magdy  | Computer Archetecture           ✅ Professor Available

VALIDATION REPORT:
✅ All day off constraints satisfied!

Schedule Statistics:
  Total instructional hours: 1.5
  Subjects scheduled: 1
  Time slots utilized: 2


====================================================================================

# With Time preference 

In [4]:
from constraint import Problem

def get_user_input():
    """Dynamically get all input data from the user"""
    subjects = []
    rooms = []
    
    print("=== University Schedule Generator - Data Input ===")
    print("\nStep 1: Enter Subject Information")
    print("-----------------------------------")
    
    # Time preference options for user reference
    time_preference_options = {
        '1': {'name': 'Morning only (9:00-12:00)', 'constraint': 'morning'},
        '2': {'name': 'Afternoon only (12:00-15:00)', 'constraint': 'afternoon'},
        '3': {'name': 'No preference', 'constraint': 'any'},
        '4': {'name': 'Before 11:00', 'constraint': 'before_11'},
        '5': {'name': 'After 11:00', 'constraint': 'after_11'}
    }
    
    while True:
        print(f"\nSubject #{len(subjects) + 1}:")
        subject_id = input("Subject ID (e.g., CS101): ").strip()
        subject_name = input("Subject Name: ").strip()
        students = int(input("Number of students: "))
        professor = input("Professor name: ").strip()
        required_room_type = input("Required room type (lab/lecture_hall): ").strip()
        day_off = input("Professor's day off (Sun/Mon/Tue/Wed/Thu or leave blank if none): ").strip()
        
        # Convert day off to proper case to match time slots
        if day_off:
            day_off = day_off.capitalize()  # Convert 'thu' to 'Thu'
        
        # Get time preference
        print("\nProfessor Time Preference Options:")
        for key, option in time_preference_options.items():
            print(f"  {key}. {option['name']}")
        
        pref_choice = input("Enter preference number (1-5): ").strip()
        time_preference = time_preference_options.get(pref_choice, time_preference_options['3'])['constraint']
        
        subject = {
            'subject_id': subject_id,
            'subject_name': subject_name,
            'students': students,
            'professor': professor,
            'required_room_type': required_room_type,
            'day_off': day_off if day_off else None,
            'time_preference': time_preference
        }
        subjects.append(subject)
        
        more = input("\nAdd another subject? (y/n): ").strip().lower()
        if more != 'y':
            break
    
    print("\nStep 2: Enter Room Information")
    print("-----------------------------------")
    
    while True:
        print(f"\nRoom #{len(rooms) + 1}:")
        room_id = input("Room ID: ").strip()
        capacity = int(input("Room capacity: "))
        room_type = input("Room type (lab/lecture_hall): ").strip()
        
        room = {
            'room_id': room_id,
            'capacity': capacity,
            'room_type': room_type
        }
        rooms.append(room)
        
        more = input("\nAdd another room? (y/n): ").strip().lower()
        if more != 'y':
            break
    
    return subjects, rooms

def create_basic_scheduler(subjects, rooms):
    """Create scheduler with day off and time preference constraints"""
    time_slots = generate_time_slots()
    
    problem = Problem()
    
    # Add variables (each subject needs a time slot and room)
    for subject in subjects:
        suitable_rooms = [room['room_id'] for room in rooms 
                         if room['capacity'] >= subject['students']]
        
        problem.addVariable(f"{subject['subject_id']}_time", time_slots)
        problem.addVariable(f"{subject['subject_id']}_room", suitable_rooms)
    
    # CONSTRAINT 1: No professor double-booking
    professor_subjects = {}
    for subject in subjects:
        prof = subject['professor']
        if prof not in professor_subjects:
            professor_subjects[prof] = []
        professor_subjects[prof].append(subject['subject_id'])
    
    for prof, subject_ids in professor_subjects.items():
        if len(subject_ids) > 1:
            for i in range(len(subject_ids)):
                for j in range(i + 1, len(subject_ids)):
                    subj1 = subject_ids[i]
                    subj2 = subject_ids[j]
                    problem.addConstraint(
                        lambda time1, time2: time1 != time2,
                        [f"{subj1}_time", f"{subj2}_time"]
                    )
    
    # CONSTRAINT 2: No room double-booking
    subject_ids = [s['subject_id'] for s in subjects]
    for i in range(len(subject_ids)):
        for j in range(i + 1, len(subject_ids)):
            subj1 = subject_ids[i]
            subj2 = subject_ids[j]
            problem.addConstraint(
                lambda time1, time2, room1, room2: 
                    not (time1 == time2 and room1 == room2),
                [f"{subj1}_time", f"{subj2}_time", f"{subj1}_room", f"{subj2}_room"]
            )
    
    # CONSTRAINT 3: Room type constraints
    for subject in subjects:
        if subject.get('required_room_type'):
            required_type = subject['required_room_type']
            problem.addConstraint(
                lambda room_id, rt=required_type: 
                    any(room['room_type'] == rt for room in rooms if room['room_id'] == room_id),
                [f"{subject['subject_id']}_room"]
            )
    
    # CONSTRAINT 4: Professor day off constraints (FIXED - case insensitive)
    for subject in subjects:
        if subject.get('day_off'):
            day_off = subject['day_off']
            # Use case-insensitive comparison
            problem.addConstraint(
                lambda time_slot, off_day=day_off: time_slot.split('_')[0].lower() != off_day.lower(),
                [f"{subject['subject_id']}_time"]
            )
    
    # CONSTRAINT 5: Professor time preference constraints
    for subject in subjects:
        time_pref = subject.get('time_preference', 'any')
        
        if time_pref != 'any':
            problem.addConstraint(
                lambda time_slot, pref=time_pref: check_time_preference(time_slot, pref),
                [f"{subject['subject_id']}_time"]
            )
    
    return problem, time_slots

def check_time_preference(time_slot, preference):
    """Check if a time slot matches the professor's time preference"""
    time_part = time_slot.split('_')[1]
    start_time_str = time_part.split('-')[0]
    start_time = convert_time_to_float(start_time_str)
    
    if preference == 'morning':
        return start_time < 12.0
    elif preference == 'afternoon':
        return start_time >= 12.0
    elif preference == 'before_11':
        return start_time < 11.0
    elif preference == 'after_11':
        return start_time >= 11.0
    elif preference == 'any':
        return True
    else:
        return True

def convert_time_to_float(time_str):
    """Convert time string like '9:00' to float like 9.0"""
    if ':' in time_str:
        hours, minutes = map(int, time_str.split(':'))
        return hours + minutes / 60.0
    else:
        return float(time_str)

def print_schedule(solution, subjects, time_slots):
    """Print the generated schedule with all constraint information"""
    if not solution:
        print("No solution found!")
        return
    
    print("\n" + "="*80)
    print("GENERATED SCHEDULE")
    print("="*80)
    
    # Display professor constraints information
    professors_info = {}
    for subject in subjects:
        prof = subject['professor']
        if prof not in professors_info:
            professors_info[prof] = {
                'day_off': subject.get('day_off'),
                'time_preference': subject.get('time_preference', 'any')
            }
    
    if professors_info:
        print("\nProfessor Constraints:")
        for prof, constraints in professors_info.items():
            day_off_info = f" | Day off: {constraints['day_off']}" if constraints['day_off'] else " | No day off"
            pref_info = f" | Time preference: {constraints['time_preference']}" if constraints['time_preference'] != 'any' else " | No time preference"
            print(f"  {prof}:{day_off_info}{pref_info}")
    
    # Group by day for better organization
    schedule_by_day = {}
    for key, value in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            room_key = f"{subject_id}_room"
            room = solution[room_key]
            day = value.split('_')[0]
            
            if day not in schedule_by_day:
                schedule_by_day[day] = []
            
            subject_info = next(s for s in subjects if s['subject_id'] == subject_id)
            schedule_by_day[day].append({
                'time_slot': value,
                'subject': subject_info,
                'room': room,
                'duration': get_slot_duration(value)
            })
    
    # Print schedule organized by day
    for day in sorted(schedule_by_day.keys()):
        print(f"\n{day.upper()}:")
        print("-" * 60)
        
        # Sort by time
        day_schedule = sorted(schedule_by_day[day], key=lambda x: x['time_slot'])
        
        for entry in day_schedule:
            subject = entry['subject']
            prof = subject['professor']
            day_off = subject.get('day_off')
            time_pref = subject.get('time_preference', 'any')
            
            # Check constraints status (FIXED - case insensitive)
            constraints_status = []
            
            # Day off check - case insensitive
            if day_off and day_off.lower() == day.lower():
                constraints_status.append("⚠️ DAY OFF")
            elif day_off:
                constraints_status.append("✅ Available")
            else:
                constraints_status.append("✅ No day off")
            
            # Time preference check
            time_part = entry['time_slot'].split('_')[1]
            start_time_str = time_part.split('-')[0]
            start_time = convert_time_to_float(start_time_str)
            
            pref_satisfied = check_time_preference(entry['time_slot'], time_pref)
            if time_pref != 'any':
                status = "✅" if pref_satisfied else "⚠️"
                constraints_status.append(f"{status} {time_pref}")
            else:
                constraints_status.append("✅ No preference")
            
            print(f"  {time_part:<11} | "
                  f"Room: {entry['room']:<6} | "
                  f"Dr. {prof:<12} | "
                  f"{subject['subject_name']:<25} | "
                  f"{' | '.join(constraints_status)}")

# Utility functions
def get_slot_duration(slot):
    day, time_range = slot.split('_')
    start, end = time_range.split('-')
    start_h, start_m = map(int, start.split(':')) if ':' in start else (int(start), 0)
    end_h, end_m = map(int, end.split(':')) if ':' in end else (int(end), 0)
    duration = (end_h + end_m/60) - (start_h + start_m/60)
    return duration

def get_available_days(time_slots):
    return list(set(slot.split('_')[0] for slot in time_slots))

def generate_time_slots(days=['Sun', 'Mon', 'Tue', 'Wed', 'Thu']):
    time_slots = []
    start_hour = 9.0
    end_hour = 15.0
    slot_duration = 1.5
    num_slots = int((end_hour - start_hour) / slot_duration)
    
    for day in days:
        current_time = start_hour
        for i in range(num_slots):
            start_time = current_time
            end_time = current_time + slot_duration
            start_str = format_time(start_time)
            end_str = format_time(end_time)
            time_slot = f"{day}_{start_str}-{end_str}"
            time_slots.append(time_slot)
            current_time = end_time
    
    return time_slots

def format_time(hour_float):
    hours = int(hour_float)
    minutes = int((hour_float - hours) * 60)
    return f"{hours}:{minutes:02d}"

def validate_constraints(solution, subjects):
    """Validate that all constraints are satisfied (FIXED - case insensitive)"""
    day_off_violations = []
    time_pref_violations = []
    
    for key, time_slot in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            subject = next(s for s in subjects if s['subject_id'] == subject_id)
            professor = subject['professor']
            day_off = subject.get('day_off')
            time_pref = subject.get('time_preference', 'any')
            day = time_slot.split('_')[0]
            
            # Check day off constraint - case insensitive
            if day_off and day_off.lower() == day.lower():
                day_off_violations.append(f"❌ {professor} scheduled on {day} (day off) for {subject['subject_name']}")
            
            # Check time preference constraint
            if time_pref != 'any' and not check_time_preference(time_slot, time_pref):
                time_pref_violations.append(f"❌ {professor}'s time preference ({time_pref}) violated for {subject['subject_name']} at {time_slot}")
    
    print("\n" + "="*70)
    print("VALIDATION REPORT:")
    print("="*70)
    
    if day_off_violations:
        print("DAY OFF CONSTRAINT VIOLATIONS:")
        for violation in day_off_violations:
            print(f"  {violation}")
    else:
        print("✅ All day off constraints satisfied!")
    
    if time_pref_violations:
        print("\nTIME PREFERENCE CONSTRAINT VIOLATIONS:")
        for violation in time_pref_violations:
            print(f"  {violation}")
    else:
        print("✅ All time preference constraints satisfied!")

# Main execution
def main():
    print("=== University Schedule Generator ===")
    print("Now with Day Off + Time Preference Constraints!")
    print("="*60)
    
    # Get all data from user
    subjects, rooms = get_user_input()
    
    # Display input summary
    print("\n" + "="*60)
    print("INPUT SUMMARY:")
    print("="*60)
    print(f"Subjects: {len(subjects)}")
    for subject in subjects:
        day_off_info = f" | Day off: {subject['day_off']}" if subject.get('day_off') else ""
        pref_info = f" | Time preference: {subject['time_preference']}" if subject.get('time_preference') != 'any' else ""
        print(f"  - {subject['subject_id']}: {subject['subject_name']} by {subject['professor']}{day_off_info}{pref_info}")
    
    print(f"\nRooms: {len(rooms)}")
    for room in rooms:
        print(f"  - {room['room_id']}: {room['capacity']} seats ({room['room_type']})")
    
    # Create scheduler
    problem, time_slots = create_basic_scheduler(subjects, rooms)
    
    # Display scheduling information
    print(f"\nScheduling Information:")
    print(f"  Time slots generated: {len(time_slots)}")
    print(f"  Available days: {', '.join(sorted(get_available_days(time_slots)))}")
    
    # Solve the schedule
    print("\n" + "="*60)
    print("GENERATING SCHEDULE...")
    print("="*60)
    
    solution = problem.getSolution()
    
    if solution:
        print_schedule(solution, subjects, time_slots)
        validate_constraints(solution, subjects)
        
        # Statistics
        total_hours = sum(get_slot_duration(slot) for slot in solution.values() 
                         if isinstance(slot, str) and '_' in slot)
        print(f"\nSchedule Statistics:")
        print(f"  Total instructional hours: {total_hours}")
        print(f"  Subjects scheduled: {len(subjects)}")
        print(f"  Time slots utilized: {len(set(solution.values()))}")
        
    else:
        print("No feasible schedule found with the given constraints!")
        print("\nPossible reasons:")
        print("- Too many conflicting constraints (day off + time preferences)")
        print("- Not enough available time slots")
        print("- Room constraints are too restrictive")

# Run the program
if __name__ == "__main__":
    main()

=== University Schedule Generator ===
Now with Day Off + Time Preference Constraints!
=== University Schedule Generator - Data Input ===

Step 1: Enter Subject Information
-----------------------------------

Subject #1:


Subject ID (e.g., CS101):  CS201
Subject Name:  Magdy shata
Number of students:  400
Professor name:  mosad
Required room type (lab/lecture_hall):  lecture_hall
Professor's day off (Sun/Mon/Tue/Wed/Thu or leave blank if none):  Thu



Professor Time Preference Options:
  1. Morning only (9:00-12:00)
  2. Afternoon only (12:00-15:00)
  3. No preference
  4. Before 11:00
  5. After 11:00


Enter preference number (1-5):  2

Add another subject? (y/n):  y



Subject #2:


...


Room #2:


Room ID:  400
Room capacity:  550
Room type (lab/lecture_hall):  lecture_hall

Add another room? (y/n):  y



Room #3:


Room ID:  1220
Room capacity:  700
Room type (lab/lecture_hall):  lecture_hall

Add another room? (y/n):  n



INPUT SUMMARY:
Subjects: 3
  - CS201: Magdy shata by mosad | Day off: Thu | Time preference: afternoon
  - CS102: HCI by galal | Day off: Mon | Time preference: morning
  - BS 201: OR by nunu | Day off: Wed

Rooms: 3
  - 3102: 600 seats (lecture_hall)
  - 400: 550 seats (lecture_hall)
  - 1220: 700 seats (lecture_hall)

Scheduling Information:
  Time slots generated: 20
  Available days: Mon, Sun, Thu, Tue, Wed

GENERATING SCHEDULE...

GENERATED SCHEDULE

Professor Constraints:
  mosad: | Day off: Thu | Time preference: afternoon
  galal: | Day off: Mon | Time preference: morning
  nunu: | Day off: Wed | No time preference

THU:
------------------------------------------------------------
  10:30-12:00 | Room: 1220   | Dr. galal        | HCI                       | ✅ Available | ✅ morning
  13:30-15:00 | Room: 1220   | Dr. nunu         | OR                        | ✅ Available | ✅ No preference

WED:
------------------------------------------------------------
  13:30-15:00 | Room: 12

In [4]:
from constraint import Problem

def get_user_input():
    """Dynamically get all input data from the user"""
    subjects = []
    rooms = []
    
    print("=== University Schedule Generator - Data Input ===")
    print("\nStep 1: Enter Subject Information")
    print("-----------------------------------")
    
    # Time preference options for user reference
    time_preference_options = {
        '1': {'name': 'Morning only (9:00-12:00)', 'constraint': 'morning'},
        '2': {'name': 'Afternoon only (12:00-15:00)', 'constraint': 'afternoon'},
        '3': {'name': 'No preference', 'constraint': 'any'},
        '4': {'name': 'Before 11:00', 'constraint': 'before_11'},
        '5': {'name': 'After 11:00', 'constraint': 'after_11'}
    }
    
    while True:
        print(f"\nSubject #{len(subjects) + 1}:")
        subject_id = input("Subject ID (e.g., CS101): ").strip()
        subject_name = input("Subject Name: ").strip()
        students = int(input("Number of students: "))
        professor = input("Professor name: ").strip()
        required_room_type = input("Required room type (lab/lecture_hall): ").strip().lower()  # FIXED: normalize case
        day_off = input("Professor's day off (Sun/Mon/Tue/Wed/Thu or leave blank if none): ").strip()
        
        # Convert day off to proper case to match time slots
        if day_off:
            day_off = day_off.capitalize()
        
        # Get time preference
        print("\nProfessor Time Preference Options:")
        for key, option in time_preference_options.items():
            print(f"  {key}. {option['name']}")
        
        pref_choice = input("Enter preference number (1-5): ").strip()
        time_preference = time_preference_options.get(pref_choice, time_preference_options['3'])['constraint']
        
        # Normalize professor name for consistency
        professor = normalize_name(professor)
        
        subject = {
            'subject_id': subject_id.upper(),  # FIXED: normalize subject ID case
            'subject_name': subject_name,
            'students': students,
            'professor': professor,
            'required_room_type': required_room_type,  # Already normalized to lower
            'day_off': day_off if day_off else None,
            'time_preference': time_preference
        }
        subjects.append(subject)
        
        more = input("\nAdd another subject? (y/n): ").strip().lower()
        if more != 'y':
            break
    
    print("\nStep 2: Enter Room Information")
    print("-----------------------------------")
    
    while True:
        print(f"\nRoom #{len(rooms) + 1}:")
        room_id = input("Room ID: ").strip()
        capacity = int(input("Room capacity: "))
        room_type = input("Room type (lab/lecture_hall): ").strip().lower()  # FIXED: normalize case
        
        room = {
            'room_id': room_id.upper(),  # FIXED: normalize room ID case
            'capacity': capacity,
            'room_type': room_type  # Already normalized to lower
        }
        rooms.append(room)
        
        more = input("\nAdd another room? (y/n): ").strip().lower()
        if more != 'y':
            break
    
    return subjects, rooms

def normalize_name(name):
    """Normalize names to Title Case for consistency"""
    return ' '.join(word.capitalize() for word in name.split())

def create_basic_scheduler(subjects, rooms):
    """Create scheduler with complete case-insensitive handling"""
    time_slots = generate_time_slots()
    
    problem = Problem()
    
    # Add variables (each subject needs a time slot and room)
    for subject in subjects:
        suitable_rooms = [room['room_id'] for room in rooms 
                         if room['capacity'] >= subject['students']]
        
        problem.addVariable(f"{subject['subject_id']}_time", time_slots)
        problem.addVariable(f"{subject['subject_id']}_room", suitable_rooms)
    
    # CONSTRAINT 1: No professor double-booking (FIXED: case insensitive professor matching)
    professor_subjects = {}
    for subject in subjects:
        prof = subject['professor'].lower()  # FIXED: use lowercase for grouping
        if prof not in professor_subjects:
            professor_subjects[prof] = []
        professor_subjects[prof].append(subject['subject_id'])
    
    for prof, subject_ids in professor_subjects.items():
        if len(subject_ids) > 1:
            for i in range(len(subject_ids)):
                for j in range(i + 1, len(subject_ids)):
                    subj1 = subject_ids[i]
                    subj2 = subject_ids[j]
                    problem.addConstraint(
                        lambda time1, time2: time1 != time2,
                        [f"{subj1}_time", f"{subj2}_time"]
                    )
    
    # CONSTRAINT 2: No room double-booking
    subject_ids = [s['subject_id'] for s in subjects]
    for i in range(len(subject_ids)):
        for j in range(i + 1, len(subject_ids)):
            subj1 = subject_ids[i]
            subj2 = subject_ids[j]
            problem.addConstraint(
                lambda time1, time2, room1, room2: 
                    not (time1 == time2 and room1 == room2),
                [f"{subj1}_time", f"{subj2}_time", f"{subj1}_room", f"{subj2}_room"]
            )
    
    # CONSTRAINT 3: Room type constraints (FIXED: case insensitive)
    for subject in subjects:
        if subject.get('required_room_type'):
            required_type = subject['required_room_type'].lower()  # FIXED: ensure lowercase
            problem.addConstraint(
                lambda room_id, rt=required_type: 
                    any(room['room_type'].lower() == rt for room in rooms if room['room_id'] == room_id),  # FIXED: case insensitive
                [f"{subject['subject_id']}_room"]
            )
    
    # CONSTRAINT 4: Professor day off constraints (FIXED: case insensitive)
    for subject in subjects:
        if subject.get('day_off'):
            day_off = subject['day_off']
            problem.addConstraint(
                lambda time_slot, off_day=day_off: time_slot.split('_')[0].lower() != off_day.lower(),
                [f"{subject['subject_id']}_time"]
            )
    
    # CONSTRAINT 5: Professor time preference constraints
    for subject in subjects:
        time_pref = subject.get('time_preference', 'any')
        
        if time_pref != 'any':
            problem.addConstraint(
                lambda time_slot, pref=time_pref: check_time_preference(time_slot, pref),
                [f"{subject['subject_id']}_time"]
            )
    
    return problem, time_slots

def check_time_preference(time_slot, preference):
    """Check if a time slot matches the professor's time preference"""
    time_part = time_slot.split('_')[1]
    start_time_str = time_part.split('-')[0]
    start_time = convert_time_to_float(start_time_str)
    
    if preference == 'morning':
        return start_time < 12.0
    elif preference == 'afternoon':
        return start_time >= 12.0
    elif preference == 'before_11':
        return start_time < 11.0
    elif preference == 'after_11':
        return start_time >= 11.0
    elif preference == 'any':
        return True
    else:
        return True

def convert_time_to_float(time_str):
    """Convert time string like '9:00' to float like 9.0"""
    if ':' in time_str:
        hours, minutes = map(int, time_str.split(':'))
        return hours + minutes / 60.0
    else:
        return float(time_str)

def print_schedule(solution, subjects, time_slots):
    """Print the generated schedule with consistent formatting"""
    if not solution:
        print("No solution found!")
        return
    
    print("\n" + "="*80)
    print("GENERATED SCHEDULE")
    print("="*80)
    
    # Display professor constraints information
    professors_info = {}
    for subject in subjects:
        prof = subject['professor']
        if prof not in professors_info:
            professors_info[prof] = {
                'day_off': subject.get('day_off'),
                'time_preference': subject.get('time_preference', 'any')
            }
    
    if professors_info:
        print("\nProfessor Constraints:")
        for prof, constraints in professors_info.items():
            day_off_info = f" | Day off: {constraints['day_off']}" if constraints['day_off'] else " | No day off"
            pref_info = f" | Time preference: {constraints['time_preference']}" if constraints['time_preference'] != 'any' else " | No time preference"
            print(f"  {prof}:{day_off_info}{pref_info}")
    
    # Group by day for better organization
    schedule_by_day = {}
    for key, value in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            room_key = f"{subject_id}_room"
            room = solution[room_key]
            day = value.split('_')[0]
            
            if day not in schedule_by_day:
                schedule_by_day[day] = []
            
            subject_info = next(s for s in subjects if s['subject_id'] == subject_id)
            schedule_by_day[day].append({
                'time_slot': value,
                'subject': subject_info,
                'room': room,
                'duration': get_slot_duration(value)
            })
    
    # Print schedule organized by day
    for day in sorted(schedule_by_day.keys()):
        print(f"\n{day.upper()}:")
        print("-" * 60)
        
        # Sort by time
        day_schedule = sorted(schedule_by_day[day], key=lambda x: x['time_slot'])
        
        for entry in day_schedule:
            subject = entry['subject']
            prof = subject['professor']
            day_off = subject.get('day_off')
            time_pref = subject.get('time_preference', 'any')
            
            # Check constraints status (FIXED: case insensitive)
            constraints_status = []
            
            # Day off check - case insensitive
            if day_off and day_off.lower() == day.lower():
                constraints_status.append("⚠️ DAY OFF")
            elif day_off:
                constraints_status.append("✅ Available")
            else:
                constraints_status.append("✅ No day off")
            
            # Time preference check
            time_part = entry['time_slot'].split('_')[1]
            start_time_str = time_part.split('-')[0]
            start_time = convert_time_to_float(start_time_str)
            
            pref_satisfied = check_time_preference(entry['time_slot'], time_pref)
            if time_pref != 'any':
                status = "✅" if pref_satisfied else "⚠️"
                constraints_status.append(f"{status} {time_pref}")
            else:
                constraints_status.append("✅ No preference")
            
            # FIXED: Consistent professor name formatting
            display_prof = f"Dr. {prof}" if not prof.lower().startswith('dr.') else prof
            print(f"  {time_part:<11} | "
                  f"Room: {entry['room']:<6} | "
                  f"{display_prof:<15} | "
                  f"{subject['subject_name']:<25} | "
                  f"{' | '.join(constraints_status)}")

# Utility functions
def get_slot_duration(slot):
    day, time_range = slot.split('_')
    start, end = time_range.split('-')
    start_h, start_m = map(int, start.split(':')) if ':' in start else (int(start), 0)
    end_h, end_m = map(int, end.split(':')) if ':' in end else (int(end), 0)
    duration = (end_h + end_m/60) - (start_h + start_m/60)
    return duration

def get_available_days(time_slots):
    return list(set(slot.split('_')[0] for slot in time_slots))

def generate_time_slots(days=['Sun', 'Mon', 'Tue', 'Wed', 'Thu']):
    time_slots = []
    start_hour = 9.0
    end_hour = 15.0
    slot_duration = 1.5
    num_slots = int((end_hour - start_hour) / slot_duration)
    
    for day in days:
        current_time = start_hour
        for i in range(num_slots):
            start_time = current_time
            end_time = current_time + slot_duration
            start_str = format_time(start_time)
            end_str = format_time(end_time)
            time_slot = f"{day}_{start_str}-{end_str}"
            time_slots.append(time_slot)
            current_time = end_time
    
    return time_slots

def format_time(hour_float):
    hours = int(hour_float)
    minutes = int((hour_float - hours) * 60)
    return f"{hours}:{minutes:02d}"

def validate_constraints(solution, subjects):
    """Validate that all constraints are satisfied (FIXED: case insensitive)"""
    day_off_violations = []
    time_pref_violations = []
    
    for key, time_slot in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            subject = next(s for s in subjects if s['subject_id'] == subject_id)
            professor = subject['professor']
            day_off = subject.get('day_off')
            time_pref = subject.get('time_preference', 'any')
            day = time_slot.split('_')[0]
            
            # Check day off constraint - case insensitive
            if day_off and day_off.lower() == day.lower():
                day_off_violations.append(f"❌ {professor} scheduled on {day} (day off) for {subject['subject_name']}")
            
            # Check time preference constraint
            if time_pref != 'any' and not check_time_preference(time_slot, time_pref):
                time_pref_violations.append(f"❌ {professor}'s time preference ({time_pref}) violated for {subject['subject_name']} at {time_slot}")
    
    print("\n" + "="*70)
    print("VALIDATION REPORT:")
    print("="*70)
    
    if day_off_violations:
        print("DAY OFF CONSTRAINT VIOLATIONS:")
        for violation in day_off_violations:
            print(f"  {violation}")
    else:
        print("✅ All day off constraints satisfied!")
    
    if time_pref_violations:
        print("\nTIME PREFERENCE CONSTRAINT VIOLATIONS:")
        for violation in time_pref_violations:
            print(f"  {violation}")
    else:
        print("✅ All time preference constraints satisfied!")

# Main execution
def main():
    print("=== University Schedule Generator ===")
    print("Now with Complete Case-Insensitive Handling!")
    print("="*60)
    
    # Get all data from user
    subjects, rooms = get_user_input()
    
    # Display input summary
    print("\n" + "="*60)
    print("INPUT SUMMARY:")
    print("="*60)
    print(f"Subjects: {len(subjects)}")
    for subject in subjects:
        day_off_info = f" | Day off: {subject['day_off']}" if subject.get('day_off') else ""
        pref_info = f" | Time preference: {subject['time_preference']}" if subject.get('time_preference') != 'any' else ""
        print(f"  - {subject['subject_id']}: {subject['subject_name']} by {subject['professor']}{day_off_info}{pref_info}")
    
    print(f"\nRooms: {len(rooms)}")
    for room in rooms:
        print(f"  - {room['room_id']}: {room['capacity']} seats ({room['room_type']})")
    
    # Create scheduler
    problem, time_slots = create_basic_scheduler(subjects, rooms)
    
    # Display scheduling information
    print(f"\nScheduling Information:")
    print(f"  Time slots generated: {len(time_slots)}")
    print(f"  Available days: {', '.join(sorted(get_available_days(time_slots)))}")
    
    # Solve the schedule
    print("\n" + "="*60)
    print("GENERATING SCHEDULE...")
    print("="*60)
    
    solution = problem.getSolution()
    
    if solution:
        print_schedule(solution, subjects, time_slots)
        validate_constraints(solution, subjects)
        
        # Statistics
        total_hours = sum(get_slot_duration(slot) for slot in solution.values() 
                         if isinstance(slot, str) and '_' in slot)
        print(f"\nSchedule Statistics:")
        print(f"  Total instructional hours: {total_hours}")
        print(f"  Subjects scheduled: {len(subjects)}")
        print(f"  Time slots utilized: {len(set(solution.values()))}")
        
    else:
        print("No feasible schedule found with the given constraints!")
        print("\nPossible reasons:")
        print("- Too many conflicting constraints")
        print("- Not enough available time slots")
        print("- Room constraints are too restrictive")

# Run the program
if __name__ == "__main__":
    main()

=== University Schedule Generator ===
Now with Complete Case-Insensitive Handling!
=== University Schedule Generator - Data Input ===

Step 1: Enter Subject Information
-----------------------------------

Subject #1:


Subject ID (e.g., CS101):  CS402
Subject Name:  Security
Number of students:  200
Professor name:  Marwa
Required room type (lab/lecture_hall):  lecture_hall
Professor's day off (Sun/Mon/Tue/Wed/Thu or leave blank if none):  Mon



Professor Time Preference Options:
  1. Morning only (9:00-12:00)
  2. Afternoon only (12:00-15:00)
  3. No preference
  4. Before 11:00
  5. After 11:00


Enter preference number (1-5):  1

Add another subject? (y/n):  y



Subject #2:


...

Room ID:  3101
Room capacity:  70
Room type (lab/lecture_hall):  lecture_hall

Add another room? (y/n):  y



Room #5:


Room ID:  3211
Room capacity:  80
Room type (lab/lecture_hall):  lecture_hall

Add another room? (y/n):  n



INPUT SUMMARY:
Subjects: 3
  - CS402: Security by Marwa | Day off: Mon | Time preference: morning
  - CS401: Artificial Intelligence by Boss | Day off: Thu | Time preference: after_11
  - CS403: Machine learning by Sheriff | Day off: Sun | Time preference: afternoon

Rooms: 5
  - 1220: 350 seats (lecture_hall)
  - 3214: 350 seats (lecture_hall)
  - 3102: 120 seats (lecture_hall)
  - 3101: 70 seats (lecture_hall)
  - 3211: 80 seats (lecture_hall)


ValueError: Domain is empty

In [2]:
from constraint import Problem
import random

def get_user_input():
    """Dynamically get all input data from the user"""
    subjects = []
    rooms = []
    
    print("=== University Schedule Generator - Data Input ===")
    print("\nStep 1: Enter Subject Information")
    print("-----------------------------------")
    
    time_preference_options = {
        '1': {'name': 'Morning only (9:00-12:00)', 'constraint': 'morning'},
        '2': {'name': 'Afternoon only (12:00-15:00)', 'constraint': 'afternoon'},
        '3': {'name': 'No preference', 'constraint': 'any'},
        '4': {'name': 'Before 11:00', 'constraint': 'before_11'},
        '5': {'name': 'After 11:00', 'constraint': 'after_11'}
    }
    
    while True:
        print(f"\nSubject #{len(subjects) + 1}:")
        subject_id = input("Subject ID (e.g., CS101): ").strip()
        subject_name = input("Subject Name: ").strip()
        students = int(input("Number of students: "))
        professor = input("Professor name: ").strip()
        required_room_type = input("Required room type (lab/lecture_hall): ").strip().lower()
        day_off = input("Professor's day off (Sun/Mon/Tue/Wed/Thu or leave blank if none): ").strip()
        
        if day_off:
            day_off = day_off.capitalize()
        
        print("\nProfessor Time Preference Options:")
        for key, option in time_preference_options.items():
            print(f"  {key}. {option['name']}")
        
        pref_choice = input("Enter preference number (1-5): ").strip()
        time_preference = time_preference_options.get(pref_choice, time_preference_options['3'])['constraint']
        
        professor = normalize_name(professor)
        
        subject = {
            'subject_id': subject_id.upper(),
            'subject_name': subject_name,
            'students': students,
            'professor': professor,
            'required_room_type': required_room_type,
            'day_off': day_off if day_off else None,
            'time_preference': time_preference
        }
        subjects.append(subject)
        
        more = input("\nAdd another subject? (y/n): ").strip().lower()
        if more != 'y':
            break
    
    print("\nStep 2: Enter Room Information")
    print("-----------------------------------")
    
    while True:
        print(f"\nRoom #{len(rooms) + 1}:")
        room_id = input("Room ID: ").strip()
        capacity = int(input("Room capacity: "))
        room_type = input("Room type (lab/lecture_hall): ").strip().lower()
        
        room = {
            'room_id': room_id.upper(),
            'capacity': capacity,
            'room_type': room_type
        }
        rooms.append(room)
        
        more = input("\nAdd another room? (y/n): ").strip().lower()
        if more != 'y':
            break
    
    return subjects, rooms

def normalize_name(name):
    """Normalize names to Title Case for consistency"""
    return ' '.join(word.capitalize() for word in name.split())

def create_basic_scheduler(subjects, rooms):
    """Create scheduler with improved constraint handling"""
    time_slots = generate_time_slots()
    
    # Shuffle time slots to avoid Thursday bias
    random.shuffle(time_slots)
    
    problem = Problem()
    
    # DEBUG: Print available resources
    print(f"\nDEBUG: Available Resources")
    print(f"Time slots: {len(time_slots)}")
    print(f"Rooms: {len(rooms)}")
    print(f"Subjects: {len(subjects)}")
    
    # Add variables with better domain filtering
    for subject in subjects:
        # More flexible room filtering - allow some overflow or find best fit
        suitable_rooms = [room['room_id'] for room in rooms 
                         if room['capacity'] >= subject['students'] * 0.8]  # Allow 80% capacity as minimum
        
        if not suitable_rooms:
            # If no perfect fit, find the closest room
            suitable_rooms = [min(rooms, key=lambda r: abs(r['capacity'] - subject['students']))['room_id']]
            print(f"Warning: No perfect room fit for {subject['subject_id']}. Using closest match.")
        
        # Filter time slots based on constraints upfront
        available_slots = time_slots.copy()
        
        # Apply day off constraint
        if subject.get('day_off'):
            available_slots = [slot for slot in available_slots 
                             if slot.split('_')[0].lower() != subject['day_off'].lower()]
        
        # Apply time preference constraint
        if subject.get('time_preference') != 'any':
            available_slots = [slot for slot in available_slots 
                             if check_time_preference(slot, subject['time_preference'])]
        
        if not available_slots:
            print(f"Warning: No available time slots for {subject['subject_id']} after constraints")
            available_slots = time_slots  # Fallback to all slots
        
        problem.addVariable(f"{subject['subject_id']}_time", available_slots)
        problem.addVariable(f"{subject['subject_id']}_room", suitable_rooms)
        
        print(f"DEBUG: {subject['subject_id']} - {len(available_slots)} time slots, {len(suitable_rooms)} rooms")
    
    # CONSTRAINT 1: No professor double-booking (relaxed)
    professor_subjects = {}
    for subject in subjects:
        prof = subject['professor'].lower()
        if prof not in professor_subjects:
            professor_subjects[prof] = []
        professor_subjects[prof].append(subject['subject_id'])
    
    for prof, subject_ids in professor_subjects.items():
        if len(subject_ids) > 1:
            print(f"DEBUG: Professor {prof} teaches {len(subject_ids)} subjects")
            for i in range(len(subject_ids)):
                for j in range(i + 1, len(subject_ids)):
                    subj1 = subject_ids[i]
                    subj2 = subject_ids[j]
                    problem.addConstraint(
                        lambda time1, time2: time1 != time2,
                        [f"{subj1}_time", f"{subj2}_time"]
                    )
    
    # CONSTRAINT 2: No room double-booking (essential)
    subject_ids = [s['subject_id'] for s in subjects]
    for i in range(len(subject_ids)):
        for j in range(i + 1, len(subject_ids)):
            subj1 = subject_ids[i]
            subj2 = subject_ids[j]
            problem.addConstraint(
                lambda time1, time2, room1, room2: 
                    not (time1 == time2 and room1 == room2),
                [f"{subj1}_time", f"{subj2}_time", f"{subj1}_room", f"{subj2}_room"]
            )
    
    # CONSTRAINT 3: Room type constraints (relaxed - make it a preference rather than hard constraint)
    for subject in subjects:
        if subject.get('required_room_type'):
            required_type = subject['required_room_type'].lower()
            # Make this a soft constraint - prefer but don't require
            problem.addConstraint(
                lambda room_id, rt=required_type: 
                    any(room['room_type'].lower() == rt for room in rooms if room['room_id'] == room_id),
                [f"{subject['subject_id']}_room"]
            )
    
    return problem, time_slots

def create_fallback_scheduler(subjects, rooms):
    """Simplified scheduler with fewer constraints for when main scheduler fails"""
    time_slots = generate_time_slots()
    random.shuffle(time_slots)  # Avoid bias
    
    problem = Problem()
    
    print("\nTrying fallback scheduler with relaxed constraints...")
    
    # Only essential constraints
    for subject in subjects:
        # Very relaxed room filtering
        suitable_rooms = [room['room_id'] for room in rooms 
                         if room['capacity'] >= subject['students'] * 0.5]  # Only 50% capacity required
        
        if not suitable_rooms:
            suitable_rooms = [room['room_id'] for room in rooms]  # Use any room
        
        problem.addVariable(f"{subject['subject_id']}_time", time_slots)
        problem.addVariable(f"{subject['subject_id']}_room", suitable_rooms)
    
    # Only essential constraint: no room double-booking
    subject_ids = [s['subject_id'] for s in subjects]
    for i in range(len(subject_ids)):
        for j in range(i + 1, len(subject_ids)):
            subj1 = subject_ids[i]
            subj2 = subject_ids[j]
            problem.addConstraint(
                lambda time1, time2, room1, room2: 
                    not (time1 == time2 and room1 == room2),
                [f"{subj1}_time", f"{subj2}_time", f"{subj1}_room", f"{subj2}_room"]
            )
    
    return problem, time_slots

def check_time_preference(time_slot, preference):
    """Check if a time slot matches the professor's time preference"""
    time_part = time_slot.split('_')[1]
    start_time_str = time_part.split('-')[0]
    start_time = convert_time_to_float(start_time_str)
    
    if preference == 'morning':
        return start_time < 12.0
    elif preference == 'afternoon':
        return start_time >= 12.0
    elif preference == 'before_11':
        return start_time < 11.0
    elif preference == 'after_11':
        return start_time >= 11.0
    elif preference == 'any':
        return True
    else:
        return True

def convert_time_to_float(time_str):
    """Convert time string like '9:00' to float like 9.0"""
    if ':' in time_str:
        hours, minutes = map(int, time_str.split(':'))
        return hours + minutes / 60.0
    else:
        return float(time_str)

def print_schedule(solution, subjects, time_slots):
    """Print the generated schedule"""
    if not solution:
        print("No solution found!")
        return
    
    print("\n" + "="*80)
    print("GENERATED SCHEDULE")
    print("="*80)
    
    # Group by day for better organization
    schedule_by_day = {}
    for key, value in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            room_key = f"{subject_id}_room"
            room = solution[room_key]
            day = value.split('_')[0]
            
            if day not in schedule_by_day:
                schedule_by_day[day] = []
            
            subject_info = next(s for s in subjects if s['subject_id'] == subject_id)
            schedule_by_day[day].append({
                'time_slot': value,
                'subject': subject_info,
                'room': room,
                'duration': get_slot_duration(value)
            })
    
    # Print schedule organized by day
    for day in sorted(schedule_by_day.keys()):
        print(f"\n{day.upper()}:")
        print("-" * 60)
        
        # Sort by time
        day_schedule = sorted(schedule_by_day[day], key=lambda x: x['time_slot'])
        
        for entry in day_schedule:
            subject = entry['subject']
            prof = subject['professor']
            
            display_prof = f"Dr. {prof}" if not prof.lower().startswith('dr.') else prof
            time_part = entry['time_slot'].split('_')[1]
            
            print(f"  {time_part:<11} | Room: {entry['room']:<6} | {display_prof:<15} | {subject['subject_name']}")

def get_slot_duration(slot):
    day, time_range = slot.split('_')
    start, end = time_range.split('-')
    start_h, start_m = map(int, start.split(':')) if ':' in start else (int(start), 0)
    end_h, end_m = map(int, end.split(':')) if ':' in end else (int(end), 0)
    duration = (end_h + end_m/60) - (start_h + start_m/60)
    return duration

def get_available_days(time_slots):
    return list(set(slot.split('_')[0] for slot in time_slots))

def generate_time_slots(days=['Sun', 'Mon', 'Tue', 'Wed', 'Thu']):
    time_slots = []
    start_hour = 9.0
    end_hour = 15.0
    slot_duration = 1.5
    num_slots = int((end_hour - start_hour) / slot_duration)
    
    # Shuffle days to avoid bias toward beginning of week
    random.shuffle(days)
    
    for day in days:
        current_time = start_hour
        for i in range(num_slots):
            start_time = current_time
            end_time = current_time + slot_duration
            start_str = format_time(start_time)
            end_str = format_time(end_time)
            time_slot = f"{day}_{start_str}-{end_str}"
            time_slots.append(time_slot)
            current_time = end_time
    
    return time_slots

def format_time(hour_float):
    hours = int(hour_float)
    minutes = int((hour_float - hours) * 60)
    return f"{hours}:{minutes:02d}"

def validate_constraints(solution, subjects):
    """Validate constraints"""
    day_off_violations = []
    time_pref_violations = []
    
    for key, time_slot in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            subject = next(s for s in subjects if s['subject_id'] == subject_id)
            professor = subject['professor']
            day_off = subject.get('day_off')
            time_pref = subject.get('time_preference', 'any')
            day = time_slot.split('_')[0]
            
            if day_off and day_off.lower() == day.lower():
                day_off_violations.append(f"❌ {professor} scheduled on {day} (day off) for {subject['subject_name']}")
            
            if time_pref != 'any' and not check_time_preference(time_slot, time_pref):
                time_pref_violations.append(f"❌ {professor}'s time preference ({time_pref}) violated for {subject['subject_name']} at {time_slot}")
    
    print("\n" + "="*70)
    print("VALIDATION REPORT:")
    print("="*70)
    
    if day_off_violations:
        print("DAY OFF CONSTRAINT VIOLATIONS:")
        for violation in day_off_violations:
            print(f"  {violation}")
    else:
        print("✅ All day off constraints satisfied!")
    
    if time_pref_violations:
        print("\nTIME PREFERENCE CONSTRAINT VIOLATIONS:")
        for violation in time_pref_violations:
            print(f"  {violation}")
    else:
        print("✅ All time preference constraints satisfied!")

def main():
    print("=== University Schedule Generator ===")
    print("Improved Version with Better Constraint Handling")
    print("="*60)
    
    # Get all data from user
    subjects, rooms = get_user_input()
    
    # Display input summary
    print("\n" + "="*60)
    print("INPUT SUMMARY:")
    print("="*60)
    print(f"Subjects: {len(subjects)}")
    for subject in subjects:
        day_off_info = f" | Day off: {subject['day_off']}" if subject.get('day_off') else ""
        pref_info = f" | Time preference: {subject['time_preference']}" if subject.get('time_preference') != 'any' else ""
        print(f"  - {subject['subject_id']}: {subject['subject_name']} by {subject['professor']}{day_off_info}{pref_info}")
    
    print(f"\nRooms: {len(rooms)}")
    for room in rooms:
        print(f"  - {room['room_id']}: {room['capacity']} seats ({room['room_type']})")
    
    # Try main scheduler first
    problem, time_slots = create_basic_scheduler(subjects, rooms)
    
    print(f"\nScheduling Information:")
    print(f"  Time slots generated: {len(time_slots)}")
    print(f"  Available days: {', '.join(sorted(get_available_days(time_slots)))}")
    
    print("\n" + "="*60)
    print("ATTEMPTING TO GENERATE SCHEDULE...")
    print("="*60)
    
    solution = None
    max_attempts = 3
    
    for attempt in range(max_attempts):
        print(f"\nAttempt {attempt + 1}/{max_attempts}...")
        
        if attempt == 0:
            # First attempt: full constraints
            problem, time_slots = create_basic_scheduler(subjects, rooms)
        else:
            # Subsequent attempts: relaxed constraints
            problem, time_slots = create_fallback_scheduler(subjects, rooms)
        
        solution = problem.getSolution()
        
        if solution:
            print(f"✅ Solution found on attempt {attempt + 1}!")
            break
        else:
            print(f"❌ No solution found with current constraint level")
    
    if solution:
        print_schedule(solution, subjects, time_slots)
        validate_constraints(solution, subjects)
        
        # Statistics
        total_hours = sum(get_slot_duration(slot) for slot in solution.values() 
                         if isinstance(slot, str) and '_' in slot)
        utilized_slots = len(set(solution.values()))
        
        print(f"\nSchedule Statistics:")
        print(f"  Total instructional hours: {total_hours}")
        print(f"  Subjects scheduled: {len(subjects)}")
        print(f"  Time slots utilized: {utilized_slots}/{len(time_slots)}")
        print(f"  Room utilization: {utilized_slots/len(rooms)*100:.1f}%")
        
    else:
        print("\n❌ No feasible schedule found after all attempts!")
        print("\nDetailed Analysis:")
        print(f"- Total time slots available: {len(time_slots)}")
        print(f"- Rooms available: {len(rooms)}")
        print(f"- Subjects to schedule: {len(subjects)}")
        print(f"- Minimum time slots needed: {len(subjects)}")
        
        # Check if basic requirements are met
        if len(time_slots) < len(subjects):
            print("❌ Problem: Not enough time slots for all subjects!")
        if len(rooms) < 1:
            print("❌ Problem: No rooms available!")
        
        # Suggest solutions
        print("\nSuggested Solutions:")
        print("1. Add more rooms or increase room capacities")
        print("2. Reduce the number of constraints (day offs, time preferences)")
        print("3. Add more available days or time slots")
        print("4. Combine subjects with the same professor")

if __name__ == "__main__":
    main()

=== University Schedule Generator ===
Improved Version with Better Constraint Handling
=== University Schedule Generator - Data Input ===

Step 1: Enter Subject Information
-----------------------------------

Subject #1:


Subject ID (e.g., CS101):  CS403
Subject Name:  Machine Learning
Number of students:  500
Professor name:  Sheriff
Required room type (lab/lecture_hall):  lecture_hall
Professor's day off (Sun/Mon/Tue/Wed/Thu or leave blank if none):  Thu



Professor Time Preference Options:
  1. Morning only (9:00-12:00)
  2. Afternoon only (12:00-15:00)
  3. No preference
  4. Before 11:00
  5. After 11:00


Enter preference number (1-5):  4

Add another subject? (y/n):  y



Subject #2:


...


Professor Time Preference Options:
  1. Morning only (9:00-12:00)
  2. Afternoon only (12:00-15:00)
  3. No preference
  4. Before 11:00
  5. After 11:00


Enter preference number (1-5):  2

Add another subject? (y/n):  n



Step 2: Enter Room Information
-----------------------------------

Room #1:


Room ID:  1220
Room capacity:  350
Room type (lab/lecture_hall):  lecture_hall

Add another room? (y/n):  3214



INPUT SUMMARY:
Subjects: 5
  - CS403: Machine Learning by Sheriff | Day off: Thu | Time preference: before_11
  - CS402: Security by Marwa | Day off: Mon | Time preference: morning
  - CS401: Artificial Intelligence by Ahmed Gaber | Day off: Wed | Time preference: after_11
  - CS417: Pattern by Ashraf | Day off: Tue | Time preference: before_11
  - CS418: Video Game by Hossam | Day off: Sun | Time preference: afternoon

Rooms: 1
  - 1220: 350 seats (lecture_hall)

DEBUG: Available Resources
Time slots: 20
Rooms: 1
Subjects: 5
DEBUG: CS403 - 8 time slots, 1 rooms
DEBUG: CS402 - 8 time slots, 1 rooms
DEBUG: CS401 - 8 time slots, 1 rooms
DEBUG: CS417 - 8 time slots, 1 rooms
DEBUG: CS418 - 8 time slots, 1 rooms

Scheduling Information:
  Time slots generated: 20
  Available days: Mon, Sun, Thu, Tue, Wed

ATTEMPTING TO GENERATE SCHEDULE...

Attempt 1/3...

DEBUG: Available Resources
Time slots: 20
Rooms: 1
Subjects: 5
DEBUG: CS403 - 8 time slots, 1 rooms
DEBUG: CS402 - 8 time slots, 1 room

In [3]:
from constraint import Problem
import random
import math

def get_user_input():
    """Dynamically get all input data from the user"""
    subjects = []
    rooms = []
    
    print("=== University Schedule Generator - Data Input ===")
    print("\nStep 1: Enter Subject Information")
    print("-----------------------------------")
    
    time_preference_options = {
        '1': {'name': 'Morning only (9:00-12:00)', 'constraint': 'morning'},
        '2': {'name': 'Afternoon only (12:00-15:00)', 'constraint': 'afternoon'},
        '3': {'name': 'No preference', 'constraint': 'any'},
        '4': {'name': 'Before 11:00', 'constraint': 'before_11'},
        '5': {'name': 'After 11:00', 'constraint': 'after_11'}
    }
    
    while True:
        print(f"\nSubject #{len(subjects) + 1}:")
        subject_id = input("Subject ID (e.g., CS101): ").strip()
        subject_name = input("Subject Name: ").strip()
        students = int(input("Number of students: "))
        professor = input("Professor name: ").strip()
        required_room_type = input("Required room type (lab/lecture_hall): ").strip().lower()
        day_off = input("Professor's day off (Sun/Mon/Tue/Wed/Thu or leave blank if none): ").strip()
        
        if day_off:
            day_off = day_off.capitalize()
        
        print("\nProfessor Time Preference Options:")
        for key, option in time_preference_options.items():
            print(f"  {key}. {option['name']}")
        
        pref_choice = input("Enter preference number (1-5): ").strip()
        time_preference = time_preference_options.get(pref_choice, time_preference_options['3'])['constraint']
        
        professor = normalize_name(professor)
        
        subject = {
            'subject_id': subject_id.upper(),
            'subject_name': subject_name,
            'students': students,
            'professor': professor,
            'required_room_type': required_room_type,
            'day_off': day_off if day_off else None,
            'time_preference': time_preference
        }
        subjects.append(subject)
        
        more = input("\nAdd another subject? (y/n): ").strip().lower()
        if more != 'y':
            break
    
    print("\nStep 2: Enter Room Information")
    print("-----------------------------------")
    
    while True:
        print(f"\nRoom #{len(rooms) + 1}:")
        room_id = input("Room ID: ").strip()
        capacity = int(input("Room capacity: "))
        room_type = input("Room type (lab/lecture_hall): ").strip().lower()
        
        room = {
            'room_id': room_id.upper(),
            'capacity': capacity,
            'room_type': room_type
        }
        rooms.append(room)
        
        more = input("\nAdd another room? (y/n): ").strip().lower()
        if more != 'y':
            break
    
    return subjects, rooms

def normalize_name(name):
    """Normalize names to Title Case for consistency"""
    return ' '.join(word.capitalize() for word in name.split())

def split_subject_if_needed(subject, rooms):
    """
    Split a subject into multiple groups if no single room can accommodate all students.
    Returns a list of subject groups.
    """
    max_room_capacity = max(room['capacity'] for room in rooms)
    
    if subject['students'] <= max_room_capacity:
        # No splitting needed
        return [subject]
    
    # Calculate how many groups we need
    groups_needed = math.ceil(subject['students'] / max_room_capacity)
    students_per_group = math.ceil(subject['students'] / groups_needed)
    
    print(f"NOTE: Subject {subject['subject_id']} has {subject['students']} students.")
    print(f"      Splitting into {groups_needed} groups of ~{students_per_group} students each.")
    
    subject_groups = []
    for i in range(groups_needed):
        group_subject = subject.copy()
        group_subject['subject_id'] = f"{subject['subject_id']}_G{i+1}"
        group_subject['students'] = students_per_group if i < groups_needed - 1 else subject['students'] - (students_per_group * (groups_needed - 1))
        group_subject['is_split_group'] = True
        group_subject['original_subject_id'] = subject['subject_id']
        group_subject['group_number'] = i + 1
        group_subject['total_groups'] = groups_needed
        subject_groups.append(group_subject)
    
    return subject_groups

def create_basic_scheduler(subjects, rooms):
    """Create scheduler with room capacity splitting"""
    time_slots = generate_time_slots()
    random.shuffle(time_slots)
    
    problem = Problem()
    
    # First, split subjects if needed
    all_subjects = []
    for subject in subjects:
        split_groups = split_subject_if_needed(subject, rooms)
        all_subjects.extend(split_groups)
    
    print(f"\nDEBUG: After splitting - Total subjects/groups: {len(all_subjects)}")
    
    # DEBUG: Print available resources
    print(f"DEBUG: Available Resources")
    print(f"Time slots: {len(time_slots)}")
    print(f"Rooms: {len(rooms)}")
    print(f"Subjects/Groups: {len(all_subjects)}")
    
    # Add variables with better domain filtering
    for subject in all_subjects:
        # Find suitable rooms for this subject/group
        suitable_rooms = [room['room_id'] for room in rooms 
                         if room['capacity'] >= subject['students']]
        
        if not suitable_rooms:
            # If no perfect fit, find the closest room
            closest_room = min(rooms, key=lambda r: abs(r['capacity'] - subject['students']))
            suitable_rooms = [closest_room['room_id']]
            print(f"Warning: No perfect room fit for {subject['subject_id']}. Using closest match: {closest_room['room_id']} (capacity: {closest_room['capacity']})")
        
        # Filter time slots based on constraints upfront
        available_slots = time_slots.copy()
        
        # Apply day off constraint
        if subject.get('day_off'):
            available_slots = [slot for slot in available_slots 
                             if slot.split('_')[0].lower() != subject['day_off'].lower()]
        
        # Apply time preference constraint
        if subject.get('time_preference') != 'any':
            available_slots = [slot for slot in available_slots 
                             if check_time_preference(slot, subject['time_preference'])]
        
        if not available_slots:
            print(f"Warning: No available time slots for {subject['subject_id']} after constraints")
            available_slots = time_slots  # Fallback to all slots
        
        problem.addVariable(f"{subject['subject_id']}_time", available_slots)
        problem.addVariable(f"{subject['subject_id']}_room", suitable_rooms)
        
        print(f"DEBUG: {subject['subject_id']} - {len(available_slots)} time slots, {len(suitable_rooms)} rooms")
    
    # CONSTRAINT 1: No professor double-booking
    professor_subjects = {}
    for subject in all_subjects:
        prof = subject['professor'].lower()
        if prof not in professor_subjects:
            professor_subjects[prof] = []
        professor_subjects[prof].append(subject['subject_id'])
    
    for prof, subject_ids in professor_subjects.items():
        if len(subject_ids) > 1:
            print(f"DEBUG: Professor {prof} teaches {len(subject_ids)} subjects/groups")
            for i in range(len(subject_ids)):
                for j in range(i + 1, len(subject_ids)):
                    subj1 = subject_ids[i]
                    subj2 = subject_ids[j]
                    problem.addConstraint(
                        lambda time1, time2: time1 != time2,
                        [f"{subj1}_time", f"{subj2}_time"]
                    )
    
    # CONSTRAINT 2: No room double-booking
    subject_ids = [s['subject_id'] for s in all_subjects]
    for i in range(len(subject_ids)):
        for j in range(i + 1, len(subject_ids)):
            subj1 = subject_ids[i]
            subj2 = subject_ids[j]
            problem.addConstraint(
                lambda time1, time2, room1, room2: 
                    not (time1 == time2 and room1 == room2),
                [f"{subj1}_time", f"{subj2}_time", f"{subj1}_room", f"{subj2}_room"]
            )
    
    # CONSTRAINT 3: Room type constraints
    for subject in all_subjects:
        if subject.get('required_room_type'):
            required_type = subject['required_room_type'].lower()
            problem.addConstraint(
                lambda room_id, rt=required_type: 
                    any(room['room_type'].lower() == rt for room in rooms if room['room_id'] == room_id),
                [f"{subject['subject_id']}_room"]
            )
    
    # NEW CONSTRAINT 4: Split subject groups must be on same day and consecutive time slots
    grouped_subjects = {}
    for subject in all_subjects:
        if subject.get('is_split_group'):
            original_id = subject['original_subject_id']
            if original_id not in grouped_subjects:
                grouped_subjects[original_id] = []
            grouped_subjects[original_id].append(subject)
    
    for original_id, groups in grouped_subjects.items():
        if len(groups) > 1:
            print(f"DEBUG: Applying constraints for split subject {original_id} with {len(groups)} groups")
            
            # Groups must be on the same day
            group_ids = [group['subject_id'] for group in groups]
            for i in range(len(group_ids)):
                for j in range(i + 1, len(group_ids)):
                    subj1 = group_ids[i]
                    subj2 = group_ids[j]
                    problem.addConstraint(
                        lambda time1, time2: time1.split('_')[0] == time2.split('_')[0],
                        [f"{subj1}_time", f"{subj2}_time"]
                    )
            
            # Groups should have consecutive time slots (as much as possible)
            # This is a soft constraint - we'll try to encourage it but not require it
            if len(groups) == 2:
                # For 2 groups, try to make them consecutive
                subj1, subj2 = group_ids[0], group_ids[1]
                problem.addConstraint(
                    lambda time1, time2: are_time_slots_consecutive(time1, time2),
                    [f"{subj1}_time", f"{subj2}_time"]
                )
    
    return problem, time_slots, all_subjects  # Return modified subjects list

def are_time_slots_consecutive(slot1, slot2):
    """Check if two time slots are consecutive on the same day"""
    if slot1.split('_')[0] != slot2.split('_')[0]:
        return False  # Different days
    
    day = slot1.split('_')[0]
    time_part1 = slot1.split('_')[1]
    time_part2 = slot2.split('_')[1]
    
    # Get all time slots for this day
    all_slots = generate_time_slots()
    day_slots = [slot for slot in all_slots if slot.startswith(day)]
    day_slots.sort()
    
    try:
        idx1 = day_slots.index(slot1)
        idx2 = day_slots.index(slot2)
        return abs(idx1 - idx2) == 1
    except ValueError:
        return False

def create_fallback_scheduler(subjects, rooms):
    """Simplified scheduler with fewer constraints for when main scheduler fails"""
    time_slots = generate_time_slots()
    random.shuffle(time_slots)
    
    problem = Problem()
    
    print("\nTrying fallback scheduler with relaxed constraints...")
    
    # Split subjects if needed
    all_subjects = []
    for subject in subjects:
        split_groups = split_subject_if_needed(subject, rooms)
        all_subjects.extend(split_groups)
    
    # Only essential constraints
    for subject in all_subjects:
        # Very relaxed room filtering
        suitable_rooms = [room['room_id'] for room in rooms 
                         if room['capacity'] >= subject['students'] * 0.5]
        
        if not suitable_rooms:
            suitable_rooms = [room['room_id'] for room in rooms]
        
        problem.addVariable(f"{subject['subject_id']}_time", time_slots)
        problem.addVariable(f"{subject['subject_id']}_room", suitable_rooms)
    
    # Only essential constraint: no room double-booking
    subject_ids = [s['subject_id'] for s in all_subjects]
    for i in range(len(subject_ids)):
        for j in range(i + 1, len(subject_ids)):
            subj1 = subject_ids[i]
            subj2 = subject_ids[j]
            problem.addConstraint(
                lambda time1, time2, room1, room2: 
                    not (time1 == time2 and room1 == room2),
                [f"{subj1}_time", f"{subj2}_time", f"{subj1}_room", f"{subj2}_room"]
            )
    
    return problem, time_slots, all_subjects

def check_time_preference(time_slot, preference):
    """Check if a time slot matches the professor's time preference"""
    time_part = time_slot.split('_')[1]
    start_time_str = time_part.split('-')[0]
    start_time = convert_time_to_float(start_time_str)
    
    if preference == 'morning':
        return start_time < 12.0
    elif preference == 'afternoon':
        return start_time >= 12.0
    elif preference == 'before_11':
        return start_time < 11.0
    elif preference == 'after_11':
        return start_time >= 11.0
    elif preference == 'any':
        return True
    else:
        return True

def convert_time_to_float(time_str):
    """Convert time string like '9:00' to float like 9.0"""
    if ':' in time_str:
        hours, minutes = map(int, time_str.split(':'))
        return hours + minutes / 60.0
    else:
        return float(time_str)

def print_schedule(solution, subjects, time_slots):
    """Print the generated schedule with grouping information"""
    if not solution:
        print("No solution found!")
        return
    
    print("\n" + "="*80)
    print("GENERATED SCHEDULE")
    print("="*80)
    
    # Group by original subject for split groups
    original_subjects = {}
    for subject in subjects:
        original_id = subject.get('original_subject_id', subject['subject_id'])
        if original_id not in original_subjects:
            original_subjects[original_id] = {
                'original_name': subject['subject_name'],
                'professor': subject['professor'],
                'total_students': 0,
                'groups': []
            }
        original_subjects[original_id]['groups'].append(subject)
        if 'is_split_group' not in subject:
            original_subjects[original_id]['total_students'] = subject['students']
        else:
            original_subjects[original_id]['total_students'] += subject['students']
    
    # Display split subject information
    split_subjects = {k: v for k, v in original_subjects.items() if len(v['groups']) > 1}
    if split_subjects:
        print("\nSPLIT SUBJECTS (Large classes divided into groups):")
        for original_id, info in split_subjects.items():
            print(f"  {original_id}: {info['original_name']} - {info['total_students']} students split into {len(info['groups'])} groups")
    
    # Group by day for better organization
    schedule_by_day = {}
    for key, value in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            room_key = f"{subject_id}_room"
            room = solution[room_key]
            day = value.split('_')[0]
            
            if day not in schedule_by_day:
                schedule_by_day[day] = []
            
            subject_info = next(s for s in subjects if s['subject_id'] == subject_id)
            schedule_by_day[day].append({
                'time_slot': value,
                'subject': subject_info,
                'room': room,
                'duration': get_slot_duration(value)
            })
    
    # Print schedule organized by day
    for day in sorted(schedule_by_day.keys()):
        print(f"\n{day.upper()}:")
        print("-" * 60)
        
        # Sort by time
        day_schedule = sorted(schedule_by_day[day], key=lambda x: x['time_slot'])
        
        for entry in day_schedule:
            subject = entry['subject']
            prof = subject['professor']
            is_split = subject.get('is_split_group', False)
            original_id = subject.get('original_subject_id', subject['subject_id'])
            
            display_prof = f"Dr. {prof}" if not prof.lower().startswith('dr.') else prof
            time_part = entry['time_slot'].split('_')[1]
            
            group_info = ""
            if is_split:
                group_info = f" [Group {subject['group_number']}/{subject['total_groups']}]"
            
            print(f"  {time_part:<11} | Room: {entry['room']:<6} | {display_prof:<15} | {subject['subject_name']}{group_info}")

def get_slot_duration(slot):
    day, time_range = slot.split('_')
    start, end = time_range.split('-')
    start_h, start_m = map(int, start.split(':')) if ':' in start else (int(start), 0)
    end_h, end_m = map(int, end.split(':')) if ':' in end else (int(end), 0)
    duration = (end_h + end_m/60) - (start_h + start_m/60)
    return duration

def get_available_days(time_slots):
    return list(set(slot.split('_')[0] for slot in time_slots))

def generate_time_slots(days=['Sun', 'Mon', 'Tue', 'Wed', 'Thu']):
    time_slots = []
    start_hour = 9.0
    end_hour = 15.0
    slot_duration = 1.5
    num_slots = int((end_hour - start_hour) / slot_duration)
    
    random.shuffle(days)
    
    for day in days:
        current_time = start_hour
        for i in range(num_slots):
            start_time = current_time
            end_time = current_time + slot_duration
            start_str = format_time(start_time)
            end_str = format_time(end_time)
            time_slot = f"{day}_{start_str}-{end_str}"
            time_slots.append(time_slot)
            current_time = end_time
    
    return time_slots

def format_time(hour_float):
    hours = int(hour_float)
    minutes = int((hour_float - hours) * 60)
    return f"{hours}:{minutes:02d}"

def validate_constraints(solution, subjects):
    """Validate constraints including split subject constraints"""
    day_off_violations = []
    time_pref_violations = []
    split_group_violations = []
    
    # Check for split subject constraints
    grouped_subjects = {}
    for subject in subjects:
        if subject.get('is_split_group'):
            original_id = subject['original_subject_id']
            if original_id not in grouped_subjects:
                grouped_subjects[original_id] = []
            grouped_subjects[original_id].append(subject)
    
    for original_id, groups in grouped_subjects.items():
        if len(groups) > 1:
            # Check if all groups are on same day
            group_times = []
            for group in groups:
                group_id = group['subject_id']
                if f"{group_id}_time" in solution:
                    group_times.append(solution[f"{group_id}_time"])
            
            days = [time.split('_')[0] for time in group_times]
            if len(set(days)) > 1:
                split_group_violations.append(f"❌ Split subject {original_id} groups are on different days")
            
            # Check if groups have consecutive time slots
            if len(groups) == 2:
                time1, time2 = group_times[0], group_times[1]
                if not are_time_slots_consecutive(time1, time2):
                    split_group_violations.append(f"⚠️ Split subject {original_id} groups are not in consecutive time slots")
    
    for key, time_slot in solution.items():
        if key.endswith('_time'):
            subject_id = key.replace('_time', '')
            subject = next(s for s in subjects if s['subject_id'] == subject_id)
            professor = subject['professor']
            day_off = subject.get('day_off')
            time_pref = subject.get('time_preference', 'any')
            day = time_slot.split('_')[0]
            
            if day_off and day_off.lower() == day.lower():
                day_off_violations.append(f"❌ {professor} scheduled on {day} (day off) for {subject['subject_name']}")
            
            if time_pref != 'any' and not check_time_preference(time_slot, time_pref):
                time_pref_violations.append(f"❌ {professor}'s time preference ({time_pref}) violated for {subject['subject_name']} at {time_slot}")
    
    print("\n" + "="*70)
    print("VALIDATION REPORT:")
    print("="*70)
    
    if day_off_violations:
        print("DAY OFF CONSTRAINT VIOLATIONS:")
        for violation in day_off_violations:
            print(f"  {violation}")
    else:
        print("✅ All day off constraints satisfied!")
    
    if time_pref_violations:
        print("\nTIME PREFERENCE CONSTRAINT VIOLATIONS:")
        for violation in time_pref_violations:
            print(f"  {violation}")
    else:
        print("✅ All time preference constraints satisfied!")
    
    if split_group_violations:
        print("\nSPLIT GROUP CONSTRAINT VIOLATIONS:")
        for violation in split_group_violations:
            print(f"  {violation}")
    else:
        print("✅ All split group constraints satisfied!")

def main():
    print("=== University Schedule Generator ===")
    print("Now with Room Capacity Splitting for Large Classes!")
    print("="*60)
    
    # Get all data from user
    subjects, rooms = get_user_input()
    
    # Display input summary
    print("\n" + "="*60)
    print("INPUT SUMMARY:")
    print("="*60)
    print(f"Subjects: {len(subjects)}")
    for subject in subjects:
        day_off_info = f" | Day off: {subject['day_off']}" if subject.get('day_off') else ""
        pref_info = f" | Time preference: {subject['time_preference']}" if subject.get('time_preference') != 'any' else ""
        print(f"  - {subject['subject_id']}: {subject['subject_name']} by {subject['professor']} ({subject['students']} students){day_off_info}{pref_info}")
    
    print(f"\nRooms: {len(rooms)}")
    for room in rooms:
        print(f"  - {room['room_id']}: {room['capacity']} seats ({room['room_type']})")
    
    # Check if any subjects need splitting
    max_capacity = max(room['capacity'] for room in rooms) if rooms else 0
    large_subjects = [s for s in subjects if s['students'] > max_capacity]
    if large_subjects:
        print(f"\nNOTE: {len(large_subjects)} subject(s) will be split into multiple groups due to room capacity limits")
        for subject in large_subjects:
            groups_needed = math.ceil(subject['students'] / max_capacity)
            print(f"  - {subject['subject_id']}: {subject['students']} students → {groups_needed} groups")
    
    # Try main scheduler first
    problem, time_slots, all_subjects = create_basic_scheduler(subjects, rooms)
    
    print(f"\nScheduling Information:")
    print(f"  Time slots generated: {len(time_slots)}")
    print(f"  Available days: {', '.join(sorted(get_available_days(time_slots)))}")
    print(f"  Total subjects/groups to schedule: {len(all_subjects)}")
    
    print("\n" + "="*60)
    print("ATTEMPTING TO GENERATE SCHEDULE...")
    print("="*60)
    
    solution = None
    max_attempts = 3
    
    for attempt in range(max_attempts):
        print(f"\nAttempt {attempt + 1}/{max_attempts}...")
        
        if attempt == 0:
            # First attempt: full constraints
            problem, time_slots, all_subjects = create_basic_scheduler(subjects, rooms)
        else:
            # Subsequent attempts: relaxed constraints
            problem, time_slots, all_subjects = create_fallback_scheduler(subjects, rooms)
        
        solution = problem.getSolution()
        
        if solution:
            print(f"✅ Solution found on attempt {attempt + 1}!")
            break
        else:
            print(f"❌ No solution found with current constraint level")
    
    if solution:
        print_schedule(solution, all_subjects, time_slots)
        validate_constraints(solution, all_subjects)
        
        # Statistics
        total_hours = sum(get_slot_duration(slot) for slot in solution.values() 
                         if isinstance(slot, str) and '_' in slot)
        utilized_slots = len(set(solution.values()))
        
        print(f"\nSchedule Statistics:")
        print(f"  Total instructional hours: {total_hours}")
        print(f"  Subjects/groups scheduled: {len(all_subjects)}")
        print(f"  Time slots utilized: {utilized_slots}/{len(time_slots)}")
        print(f"  Room utilization: {utilized_slots/len(rooms)*100:.1f}%")
        
    else:
        print("\n❌ No feasible schedule found after all attempts!")
        print("\nDetailed Analysis:")
        print(f"- Total time slots available: {len(time_slots)}")
        print(f"- Rooms available: {len(rooms)}")
        print(f"- Subjects/groups to schedule: {len(all_subjects)}")
        
        if len(time_slots) < len(all_subjects):
            print("❌ Problem: Not enough time slots for all subjects/groups!")
        
        # Suggest solutions
        print("\nSuggested Solutions:")
        print("1. Add more rooms or increase room capacities")
        print("2. Reduce the number of constraints (day offs, time preferences)")
        print("3. Add more available days or time slots")

if __name__ == "__main__":
    main()

=== University Schedule Generator ===
Now with Room Capacity Splitting for Large Classes!
=== University Schedule Generator - Data Input ===

Step 1: Enter Subject Information
-----------------------------------

Subject #1:


Subject ID (e.g., CS101):  CS403
Subject Name:  Machine Learning
Number of students:  500
Professor name:  Sheriff
Required room type (lab/lecture_hall):  lecture_lab
Professor's day off (Sun/Mon/Tue/Wed/Thu or leave blank if none):  Thu



Professor Time Preference Options:
  1. Morning only (9:00-12:00)
  2. Afternoon only (12:00-15:00)
  3. No preference
  4. Before 11:00
  5. After 11:00


Enter preference number (1-5):  1

Add another subject? (y/n):  y



Subject #2:


...


Room #4:


Room ID:  3101
Room capacity:  70
Room type (lab/lecture_hall):  lecture_hall

Add another room? (y/n):  y



Room #5:


Room ID:  3211
Room capacity:  80
Room type (lab/lecture_hall):  lecture_hall

Add another room? (y/n):  n



INPUT SUMMARY:
Subjects: 5
  - CS403: Machine Learning by Sheriff (500 students) | Day off: Thu | Time preference: morning
  - CS402: Security by Marwa (200 students) | Day off: Mon | Time preference: before_11
  - CS401: Artificial Intelligence by Ahmed Gaber (400 students) | Day off: Sun | Time preference: after_11
  - CS417: Pattern by Ashraf (250 students) | Day off: Wed | Time preference: after_11
  - CS418: Video Game by Hossam (200 students) | Day off: Thu

Rooms: 5
  - 1220: 350 seats (lecture_hall)
  - 3214: 350 seats (lecture_hall)
  - 3102: 120 seats (lecture_hall)
  - 3101: 70 seats (lecture_hall)
  - 3211: 80 seats (lecture_hall)

NOTE: 2 subject(s) will be split into multiple groups due to room capacity limits
  - CS403: 500 students → 2 groups
  - CS401: 400 students → 2 groups
NOTE: Subject CS403 has 500 students.
      Splitting into 2 groups of ~250 students each.
NOTE: Subject CS401 has 400 students.
      Splitting into 2 groups of ~200 students each.

DEBUG: After